In [157]:
import pickle,os,sys,gc
import pandas as pd
import numpy as np
import ssdeep
import glob
from tqdm import tqdm
import functools,operator
from datetime import datetime
from sklearn.metrics import *
from sklearn.metrics.pairwise import *

一個禮拜中某isp各protocols的session總數

In [46]:
isp = '台灣固網'
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"]  #改!!
file_name = "_".join(sorted(date_li)) #date_li 改!!
min_date = str(min(date_li))
max_date = str(max(date_li))

In [47]:
noise_path = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/noise_cluster.pkl'
denoise_path = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/denoise_cluster.pkl'
pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'
all_df = pickle.load(open(pickle_dir+'clusterName_overview_denoise_df_'+str(min_date)+'_'+str(date_li[-1])+'.pkl','rb'))
noise_clusters = pickle.load(open(noise_path,'rb'))
denoise_clusters = pickle.load(open(denoise_path,'rb'))
assert len(noise_clusters) + len(denoise_clusters) == len(all_df)
all_df = all_df[~all_df.idx.isin(noise_clusters)]
# all_df = all_df[all_df.idx.isin(denoise_clusters)]
# all_df = all_df.reset_index(drop=True)
all_df

,idx,timestamp,country,src_ip,ssdeep
0,20200106_15733_http,"[1578239430.546177, 1578239430.627715, 1578239...","[United States, United States, United States, ...","[66.249.69.175, 66.249.69.145, 66.249.71.5, 66...",[48:+eWa5lO918hw18VcM18xdnXtctdLJCAY6S:+eXg91u...
2,20200106_177_http,"[1578239428.145726, 1578239432.99058, 15782394...","[Netherlands, United States, Iran, United Stat...","[169.197.108.38, 216.244.66.198, 217.60.228.17...",[12:rHXq6xKv89IshSR56Y1awh5UPlVFYNjZpvhm:Da5kW...
3,20200106_51539642503_http,"[1578248664.635621, 1578248842.672464, 1578249...","[Hong Kong, Egypt, Hong Kong, China, China, Ch...","[103.90.203.174, 193.227.5.230, 103.90.203.174...",[12:7KS3TXcHnMQ7s95HfUzUfszUfsYjmHXq/Ish+j51h1...
6,20200106_136_http,"[1578239430.724066, 1578239431.97968, 15782394...","[China, China, China, Taiwan, China, China, Ch...","[106.13.108.181, 59.38.126.238, 59.38.126.238,...",[12:7QoEmHXq/I75cuXNFljhgDxKv8rDH46ZY1Zu8m:7UA...
7,20200106_2386_http,"[1578241539.855242, 1578241926.134562, 1578241...","[United States, France, France, France, France...","[108.52.243.101, 147.135.130.142, 147.135.130....",[12:7iotDpiNOmHXqzNkZr1NkWQwHj1NkZfsvq7ALTbxKv...
...,...,...,...,...,...
630,20200110_42949697374_telnet,"[1578641181.71725, 1578664950.870665]","[United States, United States]","[204.93.154.208, 204.93.154.194]","[3:ro53fg5T:ro2N, 3:ro53fg5T:ro2N, 3:ro53fg5T:..."
631,20200108_94489305705_sip,"[1578455241.326382, 1578455241.327904, 1578455...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[24:p2kQ17mDETU7ft9ckQFAOckQFA+UDR0dSRtPKiRvb3...
632,20200106_10024_sip,"[1578293821.543121, 1578293821.543636, 1578293...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[48:Yoaj0rXuXugIAO08oKY18Vje821BCkOPk/Ix:YnoXu...
633,20200106_1876_sip,"[1578239432.53756, 1578239442.108728, 15782394...","[United States, United States, United States, ...","[104.206.128.22, 104.206.128.22, 104.140.188.2...",[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...


In [48]:
def cut_protocol(cluster_id):
    '''
    GOAL: 將cluster index轉換為protocol名稱
    '''
    return cluster_id.split("_")[-1]
def count_sessions_num(time_li):
    '''
    GOAL: 依據timestamp list計算每個cluster所具有的session數量
    '''
    return len(time_li)
all_df['protocol'] = all_df.idx.map(cut_protocol)
all_df['session_num'] = all_df.timestamp.map(count_sessions_num)
all_df

,idx,timestamp,country,src_ip,ssdeep,protocol,session_num
0,20200106_15733_http,"[1578239430.546177, 1578239430.627715, 1578239...","[United States, United States, United States, ...","[66.249.69.175, 66.249.69.145, 66.249.71.5, 66...",[48:+eWa5lO918hw18VcM18xdnXtctdLJCAY6S:+eXg91u...,http,331276
2,20200106_177_http,"[1578239428.145726, 1578239432.99058, 15782394...","[Netherlands, United States, Iran, United Stat...","[169.197.108.38, 216.244.66.198, 217.60.228.17...",[12:rHXq6xKv89IshSR56Y1awh5UPlVFYNjZpvhm:Da5kW...,http,206866
3,20200106_51539642503_http,"[1578248664.635621, 1578248842.672464, 1578249...","[Hong Kong, Egypt, Hong Kong, China, China, Ch...","[103.90.203.174, 193.227.5.230, 103.90.203.174...",[12:7KS3TXcHnMQ7s95HfUzUfszUfsYjmHXq/Ish+j51h1...,http,193379
6,20200106_136_http,"[1578239430.724066, 1578239431.97968, 15782394...","[China, China, China, Taiwan, China, China, Ch...","[106.13.108.181, 59.38.126.238, 59.38.126.238,...",[12:7QoEmHXq/I75cuXNFljhgDxKv8rDH46ZY1Zu8m:7UA...,http,46402
7,20200106_2386_http,"[1578241539.855242, 1578241926.134562, 1578241...","[United States, France, France, France, France...","[108.52.243.101, 147.135.130.142, 147.135.130....",[12:7iotDpiNOmHXqzNkZr1NkWQwHj1NkZfsvq7ALTbxKv...,http,33540
...,...,...,...,...,...,...,...
630,20200110_42949697374_telnet,"[1578641181.71725, 1578664950.870665]","[United States, United States]","[204.93.154.208, 204.93.154.194]","[3:ro53fg5T:ro2N, 3:ro53fg5T:ro2N, 3:ro53fg5T:...",telnet,2
631,20200108_94489305705_sip,"[1578455241.326382, 1578455241.327904, 1578455...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[24:p2kQ17mDETU7ft9ckQFAOckQFA+UDR0dSRtPKiRvb3...,sip,3901
632,20200106_10024_sip,"[1578293821.543121, 1578293821.543636, 1578293...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[48:Yoaj0rXuXugIAO08oKY18Vje821BCkOPk/Ix:YnoXu...,sip,2277
633,20200106_1876_sip,"[1578239432.53756, 1578239442.108728, 15782394...","[United States, United States, United States, ...","[104.206.128.22, 104.206.128.22, 104.140.188.2...",[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...,sip,1917


In [49]:
print(isp,"經濾除noiseClusters後在",min_date,"至",max_date,"的期間中各protocols的session數量:")
#all_df.groupby("protocol")['timestamp'].sum().apply(len) #alternative
temp = pd.DataFrame(all_df.groupby('protocol')['session_num'].sum()).sort_values('session_num',ascending=False)
temp#.reset_index()

台灣固網 經濾除noiseClusters後在 20200106 至 20200112 的期間中各protocols的session數量:


,session_num
protocol,
ssh,9432064
smb,1864386
http,993223
smtp,131451
ftp,44616
mysql,26498
pop,9295
rpc,9150
sip,8153


跨ISP的ssdeep相似度比較與關聯

In [53]:
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','遠傳電信','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"]
protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']
min_date = str(min(date_li))
# da_prefix = +'_'+max(date_li)
root_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'
prefix = root_dir + min_date[:4]+'_'+min_date[4:6]+'_'+min_date[6:]+'/'
pickle_postfix = '_clusterID_time_country_ip_ssdeep.pkl'

all_df_li = glob.glob(prefix+'*/*/*'+pickle_postfix)

In [50]:
%%time
for i,path in enumerate(all_df_li):
    isp = path.split('/')[6]
    if i == 0 :
        all_df = pickle.load(open(path,'rb'))
        all_df['isp'] = isp
    else:
        temp = pickle.load(open(path,'rb'))
        temp['isp'] = isp
        all_df = all_df.append(temp)
all_df = all_df.reset_index(drop=True)
all_df['session_num'] = all_df.timestamp.apply(len)
all_df['protocol'] = all_df.idx.apply(lambda x:x.split('_')[-1])
all_df

CPU times: user 13.3 s, sys: 3.51 s, total: 16.9 s
Wall time: 16.9 s


,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num
0,20200106_13656_http,"[1578239437.622562, 1578239439.00718, 15782394...","[United States, United States, United States, ...","[66.249.65.188, 66.249.79.137, 66.249.71.43, 6...",[48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691uw...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",亞太電信,http,183041
1,20200106_295_http,"[1578239440.513196, 1578239443.592105, 1578239...","[Iran, Russia, United States, United States, R...","[79.132.201.185, 212.41.14.170, 216.244.66.245...",[12:rHXq6xmMVkExShI/XES+AoGY1uqh5U9Z1J3vhm:Da7...,"[(35.698, 51.4115), (56.003, 92.7993), (47.722...",亞太電信,http,81808
2,20200106_2984_http,"[1578239434.537772, 1578239437.816006, 1578239...","[Russia, None, United States, United States, U...","[5.188.84.45, 109.70.100.27, 66.249.68.16, 66....",[24:tAxtAKAFMVvoCYEQTcZMzP2tS18YH+w18YHxKt1Hqa...,"[(55.7386, 37.6068), (None, None), (37.751, -9...",亞太電信,http,64609
3,20200106_51539658953_http,"[1578243129.379452, 1578243147.033231, 1578243...","[China, China, China, China, China, China, Hon...","[103.140.126.152, 103.140.126.152, 103.140.126...",[12:72wXcHnMQ7s95HfSBBBBBBBSjmHXq/Ish+j51h1JcZ...,"[(34.7725, 113.7266), (34.7725, 113.7266), (34...",亞太電信,http,48240
4,20200106_152_http,"[1578239434.511962, 1578239437.425728, 1578239...","[China, China, China, China, China, China, Chi...","[122.14.208.63, 139.199.74.166, 148.70.163.11,...",[12:/ADXEmHXq/I75cuXNFljI4DxmMVXqH46ZY1Zu8m:zA...,"[(23.1167, 113.25), (34.7725, 113.7266), (34.7...",亞太電信,http,31897
...,...,...,...,...,...,...,...,...,...
4809,20200106_42949704991_sip,"[1578293880.656561, 1578293880.66286, 15782938...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[24:pedNtuhnz0OJdNtUJdNt7AmDED83kTibWPiF83/BZs...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",中嘉寬頻,sip,11906
4810,20200106_17559_sip,"[1578239563.410876, 1578239607.984642, 1578239...","[United States, United States, United States, ...","[104.206.128.58, 104.140.188.34, 170.130.187.3...",[12:qYlim7mDLRfAaAz85g9qX6oa+sCssVWGY07GiDv+5g...,"[(37.751, -97.822), (37.751, -97.822), (34.054...",中嘉寬頻,sip,447
4811,20200106_17467_sip,"[1578258055.286932, 1578258055.328613, 1578258...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[77.247.110.179, 77.247.110.179, 77.247.110.17...",[24:sBd8Nc7S7mDEUd1gVgjmEtKimBd8Nc7gCbd8Nc7Ke2...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",中嘉寬頻,sip,64
4812,20200109_2562_sip,"[1578577345.509497, 1578577345.723031, 1578577...","[China, China, China, China, China, China, Chi...","[115.233.218.203, 115.233.218.203, 115.233.218...",[24:qgcF8EL7mDE70a2ftSF4KinF8Eyd+UZF8EFymZ0JEC...,"[(30.294, 120.1619), (30.294, 120.1619), (30.2...",中嘉寬頻,sip,60


分protocols
把key拆成一個df
把values拆成拆成一個df
key的col去map一個function，function裡面是拿values df去map另一個function，回傳df
把df list merge by idx，挑出平均分數最高的給兩個群合併，若是最大分數=0則保留不做動作

取mean取max再取mean
1. 一個key ssdeep比較某value cluster的所有ssdeep，然後取平均 [sim_score]
    * 每個key ssdeep都可以獲得跟全部cluster相比的分數
2. 每個key ssdeep都取出分數最大的cluster id和分數 [calc_sim_score]
3. 把對應到cluster id的分數取出來計算平均
    * 同時考量數量(總和一分按比例分配)和分數再平均得到最終分數
4. 最終分數最高者進行歸類合併

In [75]:
pd.DataFrame(need_df.loc[1:,'ssdeep'])

,ssdeep
1,[24:pSAc1hnz09Ac6AcTUnQmDED8XUi1xbe/jv8XUi1EBZ...
2,[48:0U5La8En3uU5NPU5O39elU08oKY18Vje821BCkOPk/...
3,[24:sAnWPEgsp5k7mDEUd1dgF2EKimAnWPEgspYCSnWPEg...
4,[24:pSMfChnz09MfXMffnQmDED8Xzg1ge0v8Xz/BZsKiXY...
5,[24:qgOf3vI7mDL5gizTKiqf3vX4v/DRt0UUgbYMfy1wUX...
6,[24:sPLTON5F7mDEUd1TjgMRPwKimPLTON5NC9LTON51ef...
7,[48:HPpaeuP1MCJgFeU08oKY18Vje821BCkOPk/Ix:HP0e...
8,[24:qgcF8EL7mDE70a2ftSF4KinF8Eyd+UZF8EFymZ0JEC...
9,[24:pSMgZY+7mDETjO1AhLLJYFHDGodd8LMgZYOdd8LMgZ...
10,[24:pSmhnz0UKD+mDED8M13Hc8DBZsKiXYEXl8XQpkCEI:...


In [209]:
kk

,1_x,1_y,score1,score
0,,,,
NAN,3,0.0,100.0,50.0


In [224]:
score_baseline = 0

In [225]:
#要先定義score baseline，第二round至少要50以上 。第一次建議score_baseline=0就好，再跑第二次會自動以mean為baseline (至少50)
#need 3hr + 3hr

def calc_sim_score(key_hash,hashli_df):
    def sim_score(hash_li,key_hash):
        def score(value_hash,key_hash):
            return ssdeep.compare(key_hash,value_hash)
        hash_df = pd.DataFrame(hash_li)
        hash_df[1] = hash_df[0].apply(score,args=(key_hash,))
        return hash_df[1].mean()
    hashli_df['mean_score']=hashli_df['ssdeep'].apply(sim_score,args=(key_hash,))
    max_score = hashli_df['mean_score'].max()
    if max_score>score_baseline: #thr
        return [hashli_df['mean_score'].idxmax(),max_score]
    else:
        return ['NAN',max_score]

combine_dict_protocols = {} #分protocol
score_li = []
try:
    print("Now using baseline =",score_baseline)
except NameError:
    score_baseline = 0
    print("1st time running... need to run again later")
for proto in tqdm(protocols_need): #分protocol
    need_df = all_df[all_df.protocol == proto] #分protocol
    need_df = need_df.reset_index(drop=True)
    combine_dict = {} #分protocol
    for i in range(len(need_df)-1):
        key_hash_df = pd.DataFrame(need_df.loc[i,'ssdeep']) #a ssdeep string
        value_hash_df = pd.DataFrame(need_df.loc[i+1:,'ssdeep']) #list of ssdeep stings
        key_hash_df[1] = key_hash_df[0].apply(calc_sim_score,args=(value_hash_df,))
        tmp = key_hash_df[1].apply(pd.Series)
        kk1 = tmp.groupby([0]).count()
        kk2 = tmp.groupby([0]).mean()
        kk = pd.merge(kk1,kk2,left_index=True, right_index=True)
        kk['score1'] = kk['1_x']/kk['1_x'].sum()*100
        kk['score'] = (kk['1_y']+kk['score1'])/2
        combine_idx = kk['score'].idxmax()
        score = kk['score'].max()
        score_li.append(score)
        if (score>=score_baseline)and(combine_idx!='NAN'): #此key有相似的cluster可合併(thr可調整)
            combine_idx = int(combine_idx)
            now_val = set(functools.reduce(operator.iconcat, list(combine_dict.values()), []))
            if (i in now_val) and (combine_idx in now_val):
                flag = 'ic' #兩個都在。一種是兩個在一起，或是一人分處一邊
                for k,v in combine_dict.items():
                    if (i in v)and(combine_idx not in v):
                        if flag=='c': #代表在c已經合併過，要把這邊的都挪過去
                            tmp = combine_dict[key][:]
                            tmp.extend(v)
                            combine_dict[key] = tmp
                            combine_dict[k] = []
                            break
                        else: #代表首先遇到i
                            flag='i'
                            key = k
                    if (combine_idx in v)and(i not in v):
                        if flag == 'i': #代表i已經先合併過了
                            tmp = combine_dict[key][:]
                            tmp.extend(v)
                            combine_dict[key] = tmp
                            combine_dict[k] = []
                            break
                        else:
                            flag='c'
                            key = k
            elif i in now_val:
#                 flag = 'i' #僅i在
                for k,v in combine_dict.items():
                    if i in v:
                        tmp = v[:]
                        tmp.append(combine_idx)
                        combine_dict[k] = tmp
                        break
                        
            elif combine_idx in now_val:
#                 flag = 'c' #僅combine_idx在
                for k,v in combine_dict.items():
                    if combine_idx in v:
                        tmp = v[:]
                        tmp.append(i)
                        combine_dict[k]=tmp
                        break
            else: #兩個都不在
                combine_dict[i] = [i,combine_idx]
    combine_dict_protocols[proto] = combine_dict #分protocol
                
                
# #3:24:03
# need_key_li = []
# for k,v in combine_dict.items():
#     if len(v)>0:
#         print(k,len(v))
#         need_key_li.append(k)
score_baseline = np.mean(score_li) # 要跑第二次，第一次要註解掉
np.quantile(score_li,0.75),np.mean(score_li),np.quantile(score_li,0.5)        

  0%|          | 0/11 [00:00<?, ?it/s]

Now using baseline = 0


100%|██████████| 11/11 [3:15:13<00:00, 1064.91s/it]  

0 10
1 17
3 3
8 4
14 4


(97.58035714285714, 70.26529960601967, 65.61904761904762)

In [236]:
need_df

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num
383,20200106_14243_sip,"[1578239440.238951, 1578239443.45521, 15782394...","[United States, United States, United States, ...","[104.140.188.2, 104.206.128.46, 104.206.128.22...",[12:qYlim7mDLRfAaAz85g9qX6oa+sCssVWGY07GiDv+5g...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",亞太電信,sip,1146
384,20200106_77309423428_sip,"[1578320613.049779, 1578320613.105093, 1578320...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[24:pSAc1hnz09Ac6AcTUnQmDED8XUi1xbe/jv8XUi1EBZ...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",亞太電信,sip,324
385,20200106_13013_sip,"[1578314532.079033, 1578314532.114186, 1578314...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[48:0U5La8En3uU5NPU5O39elU08oKY18Vje821BCkOPk/...,"[(48.8582, 2.3387), (48.8582, 2.3387), (48.858...",亞太電信,sip,222
386,20200106_8589948585_sip,"[1578258144.368324, 1578258144.418974, 1578258...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[77.247.110.179, 77.247.110.179, 77.247.110.17...",[24:sAnWPEgsp5k7mDEUd1dgF2EKimAnWPEgspYCSnWPEg...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",亞太電信,sip,7
773,20200106_51539621821_sip,"[1578320874.953381, 1578320874.955226, 1578320...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[24:pSMfChnz09MfXMffnQmDED8Xzg1ge0v8Xz/BZsKiXY...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",中華電信,sip,9430
774,20200106_465_sip,"[1578239559.052485, 1578239573.363544, 1578239...","[United States, United States, United States, ...","[104.140.188.14, 104.140.188.14, 170.130.187.1...",[24:qgOf3vI7mDL5gizTKiqf3vX4v/DRt0UUgbYMfy1wUX...,"[(37.751, -97.822), (37.751, -97.822), (34.054...",中華電信,sip,3476
775,20200106_15702_sip,"[1578258159.385014, 1578258159.400618, 1578258...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[77.247.110.179, 77.247.110.179, 77.247.110.17...",[24:sPLTON5F7mDEUd1TjgMRPwKimPLTON5NC9LTON51ef...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",中華電信,sip,153
776,20200109_43753_sip,"[1578572318.279269, 1578572318.287146, 1578572...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[48:HPpaeuP1MCJgFeU08oKY18Vje821BCkOPk/Ix:HP0e...,"[(48.8582, 2.3387), (48.8582, 2.3387), (48.858...",中華電信,sip,66
777,20200110_34359759799_sip,"[1578641805.427443, 1578641805.450811, 1578641...","[China, China, China, China, China, China, Chi...","[202.107.226.3, 202.107.226.3, 202.107.226.3, ...",[24:qgcF8EL7mDE70a2ftSF4KinF8Eyd+UZF8EFymZ0JEC...,"[(30.294, 120.1619), (30.294, 120.1619), (30.2...",中華電信,sip,14
778,20200111_266287981719_sip,"[1578712249.318121, 1578718592.842598]","[France, France]","[5.135.73.105, 5.135.73.105]",[24:pSMgZY+7mDETjO1AhLLJYFHDGodd8LMgZYOdd8LMgZ...,"[(48.8582, 2.3387), (48.8582, 2.3387)]",中華電信,sip,2


In [238]:
agg_proto_df = {}
for proto in tqdm(protocols_need):
    combine_dict = combine_dict_protocols[proto]
    need_df = all_df[all_df.protocol == proto]
    need_df = need_df.reset_index(drop=True)
    print('running protocol:',proto)
    need_key_li = []
    for k,v in combine_dict.items():
        if len(v)>0:
            print(k,len(v))
            need_key_li.append(k)        
    agg_idxes = []
    agg_dfs = []
    for k in need_key_li:
        tmp = need_df[need_df.index.isin(combine_dict[k])]
        idx_li = tmp.idx.tolist()
        agg_idxes.extend(idx_li)
        kk = tmp.groupby(tmp.idx.isin(idx_li)).agg(list)
        kk = kk[kk.index==True] 
#         kk = kk.reset_index(drop=True)
        agg_dfs.append(kk)
#         break #接下來要再本for迴圈外面，把不存在於agg_idxes的挑出來，然後append上經group過後的df
    loner_df = need_df[~need_df.index.isin(agg_idxes)]
    for df in agg_dfs:
        loner_df = loner_df.append(df, ignore_index=True)
    agg_proto_df[proto] = loner_df
kk

  0%|          | 0/11 [00:00<?, ?it/s]

running protocol: http
0 68
1 41
3 45
4 60
5 11
7 15
8 8
9 13
10 12
11 25
12 77
16 11
18 11
20 11
21 24
24 12
26 22
28 11
29 11
30 11
35 11
38 11
39 31
40 11
42 11
45 27
49 5
52 28
55 27
56 4
57 3
58 2
59 2
60 3
63 18
64 4
65 10
66 14
73 2
74 3
75 6
76 2
83 3
85 6
105 7
107 11
111 2
115 15
118 8
130 5
133 9
134 8
140 3
143 3
147 8
153 8
155 6
157 9
159 2
160 7
169 9
170 4
176 6
178 2
181 3
195 2
265 2
288 4
347 4
348 10
377 2
389 2
435 3
439 4
469 2
478 4
547 3
552 2
563 3
567 4
581 3
584 2
612 2
613 2
618 2
621 2
637 2
643 2
661 3
678 2
681 2
707 2


  9%|▉         | 1/11 [00:02<00:21,  2.17s/it]

running protocol: mysql
0 17
1 7
2 7
3 4
7 5
10 3
12 2
14 2


 18%|█▊        | 2/11 [00:02<00:14,  1.62s/it]

...

running protocol: telnet
0 14
1 4
2 5
3 11
4 12
5 2
6 2
8 10
9 10
10 11
11 23
12 5
13 17
16 2
17 2
18 2
19 8
20 8
23 11
24 2
25 3
26 7
27 10
28 2
29 9
30 10
32 2
33 5
52 6
54 3
55 3
56 4
57 4
58 5
59 3
60 3
61 4
62 3
63 3
79 2
80 3
81 5
82 2
84 3
86 2
87 2
88 2
105 2
107 5
108 3
138 3
142 2
145 2
182 3
201 2


 91%|█████████ | 10/11 [00:46<00:06,  6.83s/it]

running protocol: sip
0 10
1 17
3 3
8 4
14 4


100%|██████████| 11/11 [00:47<00:00,  4.28s/it]


,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num
idx,,,,,,,,,
True,"[20200111_1065151919515_sip, 20200106_54116588...","[[1578672162.710047, 1578673850.835929], [1578...","[[China, Russia], [Russia, Russia], [Russia, R...","[[58.218.66.197, 193.188.22.236], [185.156.177...",[[3:OXXXL1Vf1V1V1V1RYP/LfIQfdplr+M5DcEYVV91v/v...,"[[(34.1809, 117.1569), (55.7386, 37.6068)], [(...","[台灣之星, 台灣碩網, 台固媒體, 中嘉寬頻]","[sip, sip, sip, sip]","[2, 2, 2, 9]"


In [282]:
pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/intermeidate_data/' #改
pickle.dump(obj=agg_proto_df,file=open(pickle_dir+'crossisp_ssdeep_dfdict.pkl','wb'))
print("各cluster ssdeep hash output:",pickle_dir+'crossisp_ssdeep_dfdict.pkl')

'/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/中華電信/intermeidate_data/crossisp_signature_dfdict.pkl'

1. 先把ssdeep hash攤平變成df(?)
2. 以ssdeep來做比較，算分數有大於0的就抓出來，沒過的繼續兩兩相比
3. 把至少有跟別人大於0分的都取出來後再逐一計算local alignment
4. 最後共通的順序＋文字得到結果

In [431]:
def flat_ssdeep(lioli):
    if type(lioli[0]) == list:
        return list(functools.reduce(operator.iconcat, lioli, [])) #2D to 1D list
    else:
        return lioli #only 1D list
def compare_ssdeep(hash_li):
    taken_idx = []
    need_hashes = []
    def score(value_hash,key_hash):
        return ssdeep.compare(key_hash,value_hash)
    for i,h in enumerate(hash_li):
        if i not in taken_idx: #代表我這個key還要
            compare_idx = [x+i+1 for x in range(len(hash_li)-i-1)]
            taken_idx.append(i)
            compare_idx = list(set(compare_idx) - set(taken_idx)) 
            if compare_idx == []: #發現沒人好比較了
                continue
            compare_hash = list(itemgetter(*compare_idx)(hash_li))
#             print(compare_hash,compare_idx)
            try:
                compare_df = pd.DataFrame(compare_hash,index=compare_idx)
            except ValueError:
                compare_hash = "".join(compare_hash)
                compare_df = pd.DataFrame([compare_hash],index=compare_idx)
            compare_df['score'] = compare_df[0].apply(score,args=(h,))
            compare_df_ = compare_df[compare_df['score']>0]
            taken_idx.extend(list(compare_df_.index))
            tmp = list(compare_df_[0].unique())
            tmp.append(h)
            need_hashes.extend(list(set(tmp)))
    need_hashes = sorted(need_hashes)
    return need_hashes
def get_signature(need_hashes):
    need_hashes_dict = {}
    taken_idx = []
    for i in range(len(need_hashes)):
        fn = need_hashes[i].split(":")[0]
        try: 
            if i not in taken_idx:
                keyword = need_hashes_dict[fn]
                alignments = pairwise2.align.localxx(keyword,need_hashes[i])
                li = format_alignment(*alignments[0]).split('\n')
                if int(li[3].split("=")[-1])<10: #可調整: 數字越高，signature越長、共通性越低。
                    taken_idx.append(i)
                    continue                
                need_word = []
                for i,(one,two) in enumerate(zip(li[0],li[1])):
                    if two=='|':
                        need_word.append(one)
                        try:
                            if li[1][i+1]==' ':
                                need_word.append(" ")
                        except IndexError:
                            break       
                next_compare_word = "".join(need_word)
                need_hashes_dict[fn] = next_compare_word
                taken_idx.append(i)
            else:
                continue #已經合進去過了
        except KeyError: #第一次同數字
            try:
                fn = need_hashes[i].split(":")[0]
                fn2 = need_hashes[i+1].split(":")[0]
                if ((fn == fn2)and(i not in taken_idx)and(i+1 not in taken_idx)):
                    alignments1 = pairwise2.align.localxx(need_hashes[i],need_hashes[i+1])
                    li = format_alignment(*alignments1[0]).split('\n')
                    if int(li[3].split("=")[-1])<10: #可調整: 數字越高，signature越長、共通性越低。
                        continue
                    need_word = []
                    for i,(one,two) in enumerate(zip(li[0],li[1])):
                        if two=='|':
                            need_word.append(one)
                            try:
                                if li[1][i+1]==' ':
                                    need_word.append(" ")
                            except IndexError:
                                break
                    next_compare_word = "".join(need_word)
                    need_hashes_dict[fn] = next_compare_word
                    taken_idx.extend([i,i+1])
                elif i not in taken_idx: #第一個unique fn
                    need_hashes_dict[fn] = need_hashes[i]
                    taken_idx.append(i)
                else:
                    continue
            except IndexError:
                need_hashes_dict[fn] = need_hashes[i] #沒了，最後一個unique fn
                continue
    for k,v in need_hashes_dict.items():
        if " " not in v:
            tmp = v.split(":")[1:]
            tmp =  " : ".join(kk)
            need_hashes_dict[k] = tmp.split(" ")
        else:
            tmp = v.split(" ")[1:]
            need_hashes_dict[k] = tmp
    return need_hashes_dict
            
        
# proto_df['ssdeep_flat'] = proto_df['ssdeep'].apply(flat_ssdeep)
# proto_df['ssdeep_clean'] = proto_df['ssdeep_flat'].map(compare_ssdeep)
# proto_df['signature'] = proto_df['ssdeep_clean'].map(get_signature)
# proto_df

In [432]:
# glob.glob('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/亞太電信/case_pickles/timelist_http*51539658953.pkl')
def getClusterKey_timelist(idxli,ispli):
    if type(idxli) == list: #多個cluster已經合併
        return_timeli = []
        for idx,isp in zip(idxli,ispli):
            norm = idx.split("_")
            date = norm[0]
            proto = norm[-1]
            cid = norm[1]
            base_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+date[:4]+'_'+date[4:6]+'_'+date[6:]+'/'+isp+"/case_pickles/timelist_"    #  
            pkl_path = glob.glob(base_dir+proto+"*"+cid+".pkl")
            time_li = pickle.load(open(pkl_path[0],'rb'))
            return_timeli.append(time_li)
        return return_timeli
    else: #該cluster只有自己一個人
        norm = idxli.split("_")
        date = norm[0]
        proto = norm[-1]
        cid = norm[1]   
        base_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+date[:4]+'_'+date[4:6]+'_'+date[6:]+'/'+str(ispli)+"/case_pickles/timelist_"    #  
        pkl_path = glob.glob(base_dir+proto+"*"+str(cid)+".pkl")
        time_li = pickle.load(open(pkl_path[0],'rb'))        
        return time_li
# proto_df['clusterkey_timelist'] = proto_df.apply(lambda x: getClusterKey_timelist(x.idx,x.isp)
#                                                      ,axis=1) #idx.map()    
# proto_df

In [433]:
agg_sig_df = {}
for proto in tqdm(protocols_need):
    proto_df = agg_proto_df[proto]
    proto_df['ssdeep_flat'] = proto_df['ssdeep'].apply(flat_ssdeep)
    proto_df['ssdeep_clean'] = proto_df['ssdeep_flat'].map(compare_ssdeep)
    proto_df['signature'] = proto_df['ssdeep_clean'].map(get_signature)
    proto_df['clusterkey_timelist'] = proto_df.apply(lambda x: getClusterKey_timelist(x.idx,x.isp)
                                                     ,axis=1) #idx.map()
    agg_sig_df[proto] = proto_df
gc.collect()
pickle.dump(obj=agg_sig_df,file=open(pickle_dir+"crossisp_signature_dfdict.pkl",'wb'))    
print("各isp合併後的cluster之signature及key timelist:",pickle_dir+"crossisp_signature_dfdict.pkl")
proto_df

100%|██████████| 11/11 [05:00<00:00, 27.36s/it]


個合併後的cluster之signature及key timelist: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/crossisp_signature_dfdict.pkl


,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,ssdeep_clean,signature,clusterkey_timelist
0,20200106_14243_sip,"[1578239440.238951, 1578239443.45521, 15782394...","[United States, United States, United States, ...","[104.140.188.2, 104.206.128.46, 104.206.128.22...",[12:qYlim7mDLRfAaAz85g9qX6oa+sCssVWGY07GiDv+5g...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",亞太電信,sip,1146,[12:qYlim7mDLRfAaAz85g9qX6oa+sCssVWGY07GiDv+5g...,[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...,"{'12': ['A', 'z', '5g9qX6oa+sCssVWGY07GiDv+5gT...","[1578293229.060841, 1578293229.061113, 1578293..."
1,20200106_77309423428_sip,"[1578320613.049779, 1578320613.105093, 1578320...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[24:pSAc1hnz09Ac6AcTUnQmDED8XUi1xbe/jv8XUi1EBZ...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",亞太電信,sip,324,[24:pSAc1hnz09Ac6AcTUnQmDED8XUi1xbe/jv8XUi1EBZ...,[24:pP+Jchnz0IU+JMU+JpxumDED8ePgguc8e6BZsKiXYE...,"{'24': ['mDE', 'Ki', 'XYEXl', 'Q', ':', 'h', '...","[1578320613.105093, 1578320613.105328, 1578320..."
2,20200106_13013_sip,"[1578314532.079033, 1578314532.114186, 1578314...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[48:0U5La8En3uU5NPU5O39elU08oKY18Vje821BCkOPk/...,"[(48.8582, 2.3387), (48.8582, 2.3387), (48.858...",亞太電信,sip,222,[48:0U5La8En3uU5NPU5O39elU08oKY18Vje821BCkOPk/...,[48:0U5La8En3uU5NPU5O39elU08oKY18Vje821BCkOPk/...,"{'48': ['e', 'U', '08oKY18Vje821BCkOPk/Ix:', '...","[1578314686.818419, 1578314686.818657, 1578314..."
3,20200106_8589948585_sip,"[1578258144.368324, 1578258144.418974, 1578258...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[77.247.110.179, 77.247.110.179, 77.247.110.17...",[24:sAnWPEgsp5k7mDEUd1dgF2EKimAnWPEgspYCSnWPEg...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",亞太電信,sip,7,[24:sAnWPEgsp5k7mDEUd1dgF2EKimAnWPEgspYCSnWPEg...,[24:sAnWPEgsp5k7mDEUd1dgF2EKimAnWPEgspYCSnWPEg...,"{'24': ['Egsp5k7mDEUd1', 'g', 'E', 'KimAn', 'E...","[1578258144.368324, 1578258144.368677, 1578258..."
4,20200106_51539621821_sip,"[1578320874.953381, 1578320874.955226, 1578320...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[24:pSMfChnz09MfXMffnQmDED8Xzg1ge0v8Xz/BZsKiXY...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",中華電信,sip,9430,[24:pSMfChnz09MfXMffnQmDED8Xzg1ge0v8Xz/BZsKiXY...,[24:pSMFChnz09MFXMFfnQmDED8K1PqonSjv8FBZsKiXYE...,"{'24': ['M', 'mDE', 'Y', 'g', '0']}","[1578320875.392048, 1578320875.392294, 1578320..."
5,20200106_465_sip,"[1578239559.052485, 1578239573.363544, 1578239...","[United States, United States, United States, ...","[104.140.188.14, 104.140.188.14, 170.130.187.1...",[24:qgOf3vI7mDL5gizTKiqf3vX4v/DRt0UUgbYMfy1wUX...,"[(37.751, -97.822), (37.751, -97.822), (34.054...",中華電信,sip,3476,[24:qgOf3vI7mDL5gizTKiqf3vX4v/DRt0UUgbYMfy1wUX...,[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...,"{'12': ['A', 'z', '5g9qX6oa+sCssVWGY07GiDv+5gT...","[1578240647.718236, 1578240647.718447, 1578240..."
6,20200106_15702_sip,"[1578258159.385014, 1578258159.400618, 1578258...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[77.247.110.179, 77.247.110.179, 77.247.110.17...",[24:sPLTON5F7mDEUd1TjgMRPwKimPLTON5NC9LTON51ef...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",中華電信,sip,153,[24:sPLTON5F7mDEUd1TjgMRPwKimPLTON5NC9LTON51ef...,[24:sPLTON5F7mDEUd1TjgMRPwKimPLTON5NC9LTON51ef...,"{'24': ['N5F7mDEUd1', 'g', 'KimPL', 'N5NC9L', ...","[1578258172.813086, 1578258175.811413, 1578258..."
7,20200109_43753_sip,"[1578572318.279269, 1578572318.287146, 1578572...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[48:HPpaeuP1MCJgFeU08oKY18Vje821BCkOPk/Ix:HP0e...,"[(48.8582, 2.3387), (48.8582, 2.3387), (48.858...",中華電信,sip,66,[48:HPpaeuP1MCJgFeU08oKY18Vje821BCkOPk/Ix:HP0e...,[48:HPpaeuP1MCJgFeU08oKY18Vje821BCkOPk/Ix:HP0e...,"{'48': ['e',

把每一天每一個protocol每一家isp的proto_df先吃進來放到dictionary的key(isp_日期_idx_proto)，
* iterate date(外層) 再 iterate isp(內層)來load df2
* 去逐一對應agg_sig_df iterate每個proto來看符合isp且符合date的則True
* 挑出true的再去df2裡面找time，對應protodf的session timestamp順序插入


In [13]:
agg_sig_df = pickle.load(open("/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/crossisp_signature_dfdict.pkl",'rb'))
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','遠傳電信','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"]
protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']
min_date = str(min(date_li))

In [35]:
# def get_date(idx,date):
#     if type(idx) == str:
#         time = idx.split("_")[0]
#         if time == date:
#             return True
#         else:
#             return False
#     else: #list
#         takeli = []
#         for id_ in idx:
#             if id_ == date:
#                 takeli.append(True)
#             else:
#                 takeli.append(False)
#         return takeli
def extract_timelist(idx_li , timeli,date,df2):
    """
    GOAL: get each session's packet time list
    idx_li: str or list of 20200106_14243_sip...
    timeli: list of number or list of list[[1578320613.049779, 1578320613.105093, ...
    date: an element in date_li of iteration
    df2: original df2 in preprocess phase
    """
    if type(idx_li) == str:
        time = idx_li.split("_")[0]
        if time == date:
            df2_need = df2[df2['session_time'].isin(timeli)]
            return df2_need.session_time_list.tolist()
        else:
            return False
    else: #list
        takeli = []
        for id_,times in zip(idx_li,timeli):
            if id_ == date: #此element的此list有中
                df2_need = df2[df2['session_time'].isin(times)]
                tmp = df2_need.session_time_list.tolist()
                takeli.append(tmp)
            else: #此list裡的此element沒中
                takeli.append([])
        return takeli

def extract_timelist2(idx_li , timeli,date,df2,packet_timeli):
    """
    GOAL: get each session's packet time list
    idx_li: str or list of 20200106_14243_sip...
    timeli: list of number or list of list[[1578320613.049779, 1578320613.105093, ...
    date: an element in date_li of iteration
    df2: original df2 in preprocess phase
    """
    if type(idx_li) == str:
        time = idx_li.split("_")[0]
        if time == date:
            df2_need = df2[df2['session_time'].isin(timeli)]
            return df2_need.session_time_list.tolist()
        else:
            return False
    else: #list
        takeli = []
        for id_,times,packet_li in zip(idx_li,timeli,packet_timeli):
            if id_ == date: #此element的此list有中
                df2_need = df2[df2['session_time'].isin(times)]
                tmp = df2_need.session_time_list.tolist()
                takeli.append(tmp)
            else: #此list裡的此element沒中
                takeli.append(packet_li)
        return takeli      
    
agg_sig_df2 = {}    
for i,date in tqdm(enumerate(date_li)):
    gc.collect()
    for isp in isp_li:
        dir_path = "/mnt/Raid160TB/pcap_inter/"+str(date[:4])+'_'+str(date[4:6])+'_'+str(date[6:])+"/"+isp+"/intermeidate_data/"
        df2,pickle_dir,picture_dir = pickle.load(open(dir_path+"df2_tuples_"+isp+".pkl",'rb'))
        if i==0:
            for k,df in agg_sig_df.items(): #isp覆蓋問題、有可能此protocol無恥isp問題、太久的問題
                gc.collect()
                df_tmp = df[df.isp == isp]
    #             df_tmp['take'] = df_tmp.idx.apply(get_date,args=(date,))
#                 if i==0:
                df_tmp['packet_timeli'] = df_tmp.apply(lambda x: extract_timelist(x.idx,x.timestamp,date,df2),axis=1)
#                 else:
                #merge df_tmp to df on index only packet_timeli col
                df_tmp = df.merge(pd.DataFrame(df_tmp['packet_timeli']), left_index=True, right_index=True)
                agg_sig_df2[k] = df_tmp
        elif i%2==1:
            for k2,df_2 in agg_sig_df2.items():
                df_tmp = df_2[df_2.isp == isp]
                df_tmp['packet_timeli'] = df_tmp.apply(lambda x: extract_timelist2(x.idx,x.timestamp,date,df2,x.packet_timeli),axis=1)
                agg_sig_df[k2] = df_tmp
        else:
            for k,df in agg_sig_df.items():
                df_tmp = df[df.isp == isp]
                df_tmp['packet_timeli'] = df_tmp.apply(lambda x: extract_timelist2(x.idx,x.timestamp,date,df2,x.packet_timeli),axis=1)
                agg_sig_df2[k] = df_tmp
                gc.collect()
            
df_tmp

0it [00:00, ?it/s]/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
0it [30:12:00, ?it/s]


ValueError: Wrong number of items passed 13, placement implies 1

In [46]:
# df[df.isp == isp]
isp

'遠傳電信'

In [44]:
agg_sig_df2['mysql']

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,ssdeep_clean,signature,clusterkey_timelist,packet_timeli
39,20200107_5913_mysql,"[1578328943.957562, 1578336372.190103, 1578342...","[China, China, China, France, Italy, China, Ch...","[101.251.238.52, 152.136.89.155, 101.251.238.5...","[3:gV9V1VU:gO, 3:gV9V1VU:gO, 3:OXXXL1Vf1V1V1V1...","[(34.7725, 113.7266), (34.7725, 113.7266), (34...",台灣大哥大,mysql,2406,"[3:gV9V1VU:gO, 3:gV9V1VU:gO, 3:OXXXL1Vf1V1V1V1...",[3:OXXXL1Vf1V1V1V1RYP/LfIQfdplr+M5DcEYVV91v/2v...,{'3': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafznoG...,"[1578406896.476166, 1578406896.476492, 1578406...",False
40,20200106_60129590203_mysql,"[1578247582.364033, 1578249641.875613, 1578251...","[Russia, Germany, Germany, Germany, Germany, G...","[193.188.22.234, 88.214.26.20, 88.214.26.20, 8...",[3:I3fVevXV1R13vV1V1V1V1V1V1V1V1V1V1V1V1V1V1V1...,"[(55.7386, 37.6068), (51.2993, 9.491), (51.299...",台灣大哥大,mysql,336,[3:I3fVevXV1R13vV1V1V1V1V1V1V1V1V1V1V1V1V1V1V1...,[3:DV9VevXV1R13vV1V1V1V1V1V1V1V1V1V1V1V1V1V1V1...,"{'3': ['V', 'X', 'V1', 'R13vV1V1V1V1V1V1V1V1V1...","[1578265035.50852, 1578265035.508983, 15782650...","[[1578265035.50852, 1578265035.508983, 1578265..."
41,20200107_8589934618_mysql,"[1578360438.903406, 1578379984.570746, 1578394...","[China, China, United States, China]","[101.251.238.52, 101.251.238.52, 171.67.70.88,...","[3:OXXUT8UU1V1V5WCX4VP6kfV1C2fVE:O0TJCX49fS2m,...","[(34.7725, 113.7266), (34.7725, 113.7266), (37...",台灣大哥大,mysql,4,"[3:OXXUT8UU1V1V5WCX4VP6kfV1C2fVE:O0TJCX49fS2m,...","[3:OXXUT8UU1V1V5WCX/VGl9VVfVE:O0TJCXQPm, 3:OXX...","{'3': ['V', 'V', 'fVE:O0TJCX', 'm']}","[1578379984.570746, 1578379984.570994, 1578379...",False
42,20200107_171798734513_mysql,"[1578329184.476109, 1578340394.988128, 1578359...","[China, China, China, China]","[101.251.238.52, 101.251.238.52, 101.251.238.5...","[3:11V1V1VOdC9W:QC4, 3:11V1V1VOdC9W:QC4, 3:11V...","[(34.7725, 113.7266), (34.7725, 113.7266), (34...",台灣大哥大,mysql,4,"[3:11V1V1VOdC9W:QC4, 3:11V1V1VOdC9W:QC4, 3:11V...",[3:11V1V1VOdC9W:QC4],{'3': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafznoG...,"[1578340394.988128, 1578340394.988325, 1578340...",False
43,20200107_42949716169_mysql,"[1578329815.838746, 1578356789.310997, 1578411...","[China, China, China]","[101.251.238.52, 101.251.238.52, 101.251.238.52]",[6:gH9AKrsfSJTGTKlW6sfSJaiJeVT9T7wRC3ySCb0SclV...,"[(34.7725, 113.7266), (34.7725, 113.7266), (34...",台灣大哥大,mysql,3,[6:gH9AKrsfSJTGTKlW6sfSJaiJeVT9T7wRC3ySCb0SclV...,[6:gH9AKrsfSJTGTKlW6sfSJaiJeVT9T7wRC3ySCb0SclV...,{'6': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafznoG...,"[1578411734.883605, 1578411734.883959, 1578411...",False
44,20200107_188978607177_mysql,"[1578393385.143257, 1578394444.755955]","[United States, United States]","[171.67.70.88, 171.67.70.88]",[6:sTrp06FTAqRARU22cLFVL2cLJiEfBToOFu6hdcVIiP/...,"[(37.4446, -122.1835), (37.4446, -122.1835)]",台灣大哥大,mysql,2,[6:sTrp06FTAqRARU22cLFVL2cLJiEfBToOFu6hdcVIiP/...,[6:sTrp06FTAqRARU22cLFVL2cLJiEfBToOFu6hdcVIiP/...,{'6': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafznoG...,"[1578394444.755955, 1578394444.75612, 15783944...",False
45,20200107_300647735367_mysql,"[1578393501.395347, 1578394486.787683]","[United States, United States]","[171.67.70.88, 171.67.70.88]","[3:OXXXb1ZP/LfIQfdplr+Mw9H91v/SXVV:OvPxdL+MEy,...","[(37.4446, -122.1835), (37.4446, -122.1835)]",台灣大哥大,mysql,2,"[3:OXXXb1ZP/LfIQfdplr+Mw9H91v/SXVV:OvPxdL+MEy,...",[3:OXXXb1ZP/LfIQfdplr+Mw9H91v/SXVV:OvPxdL+MEy],{'3': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafznoG...,"[1578393501.395347, 1578393501.395583, 1578393...",False


In [27]:
agg_sig_df2 = agg_sig_df
for (k1,v1),(k2,v2) in zip(agg_sig_df.items(),agg_sig_df2.items()):
    break
v2

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,signature,ssdeep_clean,clusterkey_timelist
0,20200106_13656_http,"[1578239437.622562, 1578239439.00718, 15782394...","[United States, United States, United States, ...","[66.249.65.188, 66.249.79.137, 66.249.71.43, 6...",[48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691uw...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",亞太電信,http,183041,[48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691uw...,"{'12': ['HXq', '6xmo/+R', 'Nk', 'g', 'c', 'G',...",[12:7XiS9HXq6xmo/+R+pHCFNkDTg1IEcuzGpfAvTYXNkZ...,"[1578270108.689238, 1578270108.68945, 15782701..."
1,20200106_295_http,"[1578239440.513196, 1578239443.592105, 1578239...","[Iran, Russia, United States, United States, R...","[79.132.201.185, 212.41.14.170, 216.244.66.245...",[12:rHXq6xmMVkExShI/XES+AoGY1uqh5U9Z1J3vhm:Da7...,"[(35.698, 51.4115), (56.003, 92.7993), (47.722...",亞太電信,http,81808,[12:rHXq6xmMVkExShI/XES+AoGY1uqh5U9Z1J3vhm:Da7...,"{'12': ['HX', 'x', 'I', '1', 'm:'], '24': ['n'...",[12:rHXAxmMVXo5wVfHClIshpeR56Y1awh5UPifVsy+m:D...,"[1578268769.767172, 1578268769.767411, 1578268..."
2,20200106_2984_http,"[1578239434.537772, 1578239437.816006, 1578239...","[Russia, None, United States, United States, U...","[5.188.84.45, 109.70.100.27, 66.249.68.16, 66....",[24:tAxtAKAFMVvoCYEQTcZMzP2tS18YH+w18YHxKt1Hqa...,"[(55.7386, 37.6068), (None, None), (37.751, -9...",亞太電信,http,64609,[24:tAxtAKAFMVvoCYEQTcZMzP2tS18YH+w18YHxKt1Hqa...,"{'24': ['+', 'w', '8', 'H', 'Y', ':', 'U'], '3...",[24:9SAvS18YH+w18YHYR385HltF0Vrde4+MVZMkMVHCYx...,"[1578268911.573342, 1578268911.573646, 1578268..."
3,20200106_51539658953_http,"[1578243129.379452, 1578243147.033231, 1578243...","[China, China, China, China, China, China, Hon...","[103.140.126.152, 103.140.126.152, 103.140.126...",[12:72wXcHnMQ7s95HfSBBBBBBBSjmHXq/Ish+j51h1JcZ...,"[(34.7725, 113.7266), (34.7725, 113.7266), (34...",亞太電信,http,48240,[12:72wXcHnMQ7s95HfSBBBBBBBSjmHXq/Ish+j51h1JcZ...,"{'12': ['HXq', 'x', 'm', '6', 'Z', ':', 'a'], ...",[12:+KmHXq/Ish+j51h1sXxMsA86njXAxmMVkGH46ZY1Zu...,"[1578305243.631025, 1578305243.631348, 1578305..."
4,20200106_152_http,"[1578239434.511962, 1578239437.425728, 1578239...","[China, China, China, China, China, China, Chi...","[122.14.208.63, 139.199.74.166, 148.70.163.11,...",[12:/ADXEmHXq/I75cuXNFljI4DxmMVXqH46ZY1Zu8m:zA...,"[(23.1167, 113.25), (34.7725, 113.7266), (34.7...",亞太電信,http,31897,[12:/ADXEmHXq/I75cuXNFljI4DxmMVXqH46ZY1Zu8m:zA...,"{'12': ['7', 'X', 'j', 'm', 'm:', 'M'], '24': ...",[12:/ADXEmHXq/I75cuXNFljI4DxmMVXqH46ZY1Zu8m:zA...,"[1578286833.368446, 1578286833.3688, 157828683..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,"[20200109_317827601065_http, 20200109_16320876...","[[1578545206.609212, 1578545369.126079, 157854...","[[Russia, Russia, Russia, Russia, Russia, Russ...","[[193.23.3.36, 193.23.3.36, 193.23.3.36, 193.2...",[[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA...,"[[(55.7386, 37.6068), (55.7386, 37.6068), (55....","[台灣碩網, 台灣碩網]","[http, http]","[9, 2]",[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA4...,"{'48': ['a', 'Q5jqXMz', 'A', 'q', ':', '5Rz', ...",[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA4...,"[[1578545206.609212, 1578545206.609449, 157854..."
1203,"[20200107_206158449329_http, 20200106_12025908...","[[1578333098.110814, 1578398130.044447, 157860...","[[Germany, Russia, United States, Ukraine, Ukr...","[[185.220.101.79, 5.188.210.65, 207.97.174.134...",[[96:Cdo2Hqv5BI93yS1uw1Acz5W5BI93yx5Kx:Cdo2Kv5...,"[[(51.2993, 9.491), (55.7386, 37.6068), (40.71...","[台灣碩網, 台灣碩網, 台灣碩網]","[http, http, http]","[5, 3, 3]",[96:Cdo2Hqv5BI93yS1uw1Acz5W5BI93yx5Kx:Cdo2Kv5B...,"{'96': ['B', 'I', '9', 'yS1uw1', 'B', 'I', '9'...",[96:7+y28P2BI9XyS1uw1sO5Yy28P2BI9Xyn5Kx:7+yX2B...,"[[1578333098.110814, 1578333098.111041, 157833..."
1204,"[20200109_85899367448_http, 20200109_858994174...","[[1578545199.046963, 1578565438.876521, 157856...","[[Russia, Russia, Russia], [Russia, Russia]]","[[193.23.3.36, 193.23.3.36, 193.

In [15]:
def extract_timelist(idx_li , timeli,date,df2):
    """
    GOAL: get each session's packet time list
    idx_li: str or list of 20200106_14243_sip...
    timeli: list of number or list of list[[1578320613.049779, 1578320613.105093, ...
    date: an element in date_li of iteration
    df2: original df2 in preprocess phase
    """
    if type(idx_li) == str:
        time = idx_li.split("_")[0]
        if time == date:
            df2_need = df2[df2['session_time'].isin(timeli)]
            return df2_need.session_time_list.tolist()
        else:
            return False
    else: #list
        takeli = []
        for id_,times in zip(idx_li,timeli):
            if id_ == date: #此element的此list有中
                df2_need = df2[df2['session_time'].isin(times)]
                tmp = df2_need.session_time_list.tolist()
                takeli.append(tmp)
            else: #此list裡的此element沒中
                takeli.append([])
        return takeli

def extract_timelist2(idx_li , timeli,date,df2,packet_timeli):
    """
    GOAL: get each session's packet time list
    idx_li: str or list of 20200106_14243_sip...
    timeli: list of number or list of list[[1578320613.049779, 1578320613.105093, ...
    date: an element in date_li of iteration
    df2: original df2 in preprocess phase
    """
    if type(idx_li) == str:
        time = idx_li.split("_")[0]
        if time == date:
            df2_need = df2[df2['session_time'].isin(timeli)]
            return df2_need.session_time_list.tolist()
        else:
            return False
    else: #list
        takeli = []
        for id_,times,packet_li in zip(idx_li,timeli,packet_timeli):
            if id_ == date: #此element的此list有中
                df2_need = df2[df2['session_time'].isin(times)]
                tmp = df2_need.session_time_list.tolist()
                takeli.append(tmp)
            else: #此list裡的此element沒中
                takeli.append(packet_li)
        return takeli    
i=0
for df in agg_sig_df.values():
    df_tmp = df[df.isp == isp]
#     df_tmp['take'] = df_tmp.idx.apply(get_date,args=(date,))
    if i==0:
        df_tmp['packet_timeli'] = df_tmp.apply(lambda x: extract_timelist(x.idx,x.timestamp,date,df2),axis=1)
    else:
        df_tmp['packet_timeli'] = df_tmp.apply(lambda x: extract_timelist2(x.idx,x.timestamp,date,df2,packet_timeli),axis=1)
    df = df.merge(pd.DataFrame(df_tmp['packet_timeli']), left_index=True, right_index=True)
    gc.collect()
    break

/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [16]:
df

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,signature,ssdeep_clean,clusterkey_timelist,packet_timeli
819,20200106_15666_http,"[1578239454.109651, 1578239470.109467, 1578239...","[United States, United States, United States, ...","[66.249.79.68, 66.249.75.7, 66.249.79.47, 66.2...",[48:+xfTmaK8O918hw18VcM18xdnXtctdLJCAY6S:+9Yt9...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",台固媒體,http,142441,[48:+xfTmaK8O918hw18VcM18xdnXtctdLJCAY6S:+9Yt9...,"{'48': ['a', 'K', 'O', '918hw18VcM18xdnXtctdLJ...",[48:+/aKfAO918hw18VcM18xdnXtctdLJCAY6S:+3h91uw...,"[1578323349.148714, 1578323349.148975, 1578323...","[[1578323349.148714, 1578323349.148975, 157832..."
820,20200106_701_http,"[1578239433.558292, 1578239437.866685, 1578239...","[Vietnam, United States, Iran, United States, ...","[103.130.58.27, 216.244.66.196, 5.235.204.10, ...",[12:rHXq6x5i7CIs0+R56Y1awh5UPlVFYNjZpvhm:DaKi7...,"[(10.8242, 106.6275), (47.7229, -122.1961), (3...",台固媒體,http,115957,[12:rHXq6x5i7CIs0+R56Y1awh5UPlVFYNjZpvhm:DaKi7...,"{'12': ['H', 'Xq', '1', 'm:', 'a', 'i'], '24':...",[12:7KS/cHnMQ7s95HfWfUTf5jmHXq/Is0Kj51h6Y1awh5...,"[1578293492.332239, 1578293492.332386, 1578293...","[[1578293492.332239, 1578293492.332386, 157829..."
821,20200106_1946_http,"[1578239461.751976, 1578239567.092316, 1578239...","[Chile, Chile, France, China, China, China, Fr...","[186.10.66.139, 186.10.66.139, 51.89.100.102, ...",[12:72SEmHXq/I75cuXNFljI4Dx5iWGH46ZY1Zu8m:7eAa...,"[(-33.4513, -70.6653), (-33.4513, -70.6653), (...",台固媒體,http,95856,[12:72SEmHXq/I75cuXNFljI4Dx5iWGH46ZY1Zu8m:7eAa...,"{'12': ['m', 'HXq', 'x', '5i', 'u', ':', 'A', ...",[12:5/jEmHXq/IsheY1awh5UP+/4D7eh2x5iCH46ZY1Zu8...,"[1578260839.106733, 1578260839.107064, 1578260...","[[1578260839.106733, 1578260839.107064, 157826..."
822,20200106_1947_http,"[1578239434.113271, 1578239450.110779, 1578239...","[United States, United States, United States, ...","[66.249.69.237, 66.249.79.13, 66.249.79.56, 66...",[24:uaKiWMS18YH+w18YHxKafv4qHZt1HBlxiWlbMzc9:u...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",台固媒體,http,74275,[24:uaKiWMS18YH+w18YHxKafv4qHZt1HBlxiWlbMzc9:u...,"{'12': ['Nk', 'H18YH14NIL18YH1Iy', 'SuIsheY1aw...",[12:rHXRNkH18YH14NIL18YH1IynHSuIsheY1awh5UPqox...,"[1578323862.516703, 1578323865.488656, 1578323...","[[1578323862.516703, 1578323865.488656, 157832..."
823,20200106_249108135258_http,"[1578242571.285274, 1578303819.407774, 1578325...","[United States, United States, United States, ...","[66.249.79.27, 66.249.79.232, 66.249.79.54, 66...",[24:tAACfDXMcXoaKiBZt1HyxiOfAACfDXKCTjeN1dqg5t...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",台固媒體,http,38280,[24:tAACfDXMcXoaKiBZt1HyxiOfAACfDXKCTjeN1dqg5t...,"{'24': ['cXoaKi', 't1Hyxi', 'C', 'TjeN1dqg', '...",[24:jcXoaKiBKt1HyxibvCTjeN1dqg5tfocGofALEafC6B...,"[1578303819.407774, 1578303819.408151, 1578303...","[[1578303819.407774, 1578303819.408151, 157830..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,20200106_188978582974_http,"[1578257372.233309, 1578292846.75799]","[China, China]","[106.12.69.27, 106.12.69.27]",[48:/1M2xiyTkzVhKFSVjhjjhX3KaZMYVn18hw18ut1B6C...,"[(31.0449, 121.4012), (31.0449, 121.4012)]",台固媒體,http,2,[48:/1M2xiyTkzVhKFSVjhjjhX3KaZMYVn18hw18ut1B6C...,{'24': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafzno...,[24:5dGtdU/duGPCyVJpN/jJpN/NRNUEN/aw9cOFjF:P/s...,"[1578292846.75799, 1578292846.758222, 15782928...","[[1578292846.75799, 1578292846.758222, 1578292..."
881,20200106_463856473353_http,"[1578293026.946426, 1578293034.826445]","[Iran, Iran]","[185.206.236.68, 185.206.236.68]",[12:rHXqthkVdViaUo/gYt3f3uKccx5iqB6ZYSxVfHCsm:...,"[(35.698, 51.4115), (35.698, 51.4115)]",台固媒體,http,2,[12:rHXqthkVdViaUo/gYt3f3uKccx5iqB6ZYSxVfHCsm:...,{'12': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafzno...,[12:rHXqthkVdViaUo/gYt3f3uKccx5iqB6ZYSxVfHCsm:...,"[1578293034.826445, 1578293034.826698, 1578293...","[[1578293034.826445, 1578293034.826698, 157829..."
882,20200107_360777275961_http,"[1578406062.829609, 1578406123.100342]

In [457]:
df.merge(pd.DataFrame(df_tmp['packet_timeli']), left_index=True, right_index=True)`

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,signature,ssdeep_clean,clusterkey_timelist,packet_timeli
937,20200106_23217_http,"[1578239430.609722, 1578239432.907151, 1578239...","[United States, United States, United States, ...","[66.249.64.245, 66.249.65.104, 66.249.71.53, 6...",[24:tA4ucXoajPt1HysrEA4s2XDCTjeN1dqgrFxtCBtDhL...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",台灣大哥大,http,172870,[24:tA4ucXoajPt1HysrEA4s2XDCTjeN1dqgrFxtCBtDhL...,"{'24': ['t', '1', 'C', 'T', 'N', '1', 'dqg', '...",[24:7H+cXoa+Ft1HyNd6FUNCTjeN1dqg5tfocGofALEafC...,"[1578262953.089617, 1578262953.089792, 1578262...","[[1578262953.089617, 1578262953.089792, 157826..."
938,20200106_595_http,"[1578239430.767826, 1578239432.515023, 1578239...","[Iran, China, Belarus, China, China, Philippin...","[2.183.95.130, 49.234.29.65, 178.122.68.106, 4...",[12:JkHXq/IsheY1awh5UP+/4D7eh2x7AGGH46ZY1Zu8m:...,"[(35.698, 51.4115), (34.7725, 113.7266), (53.8...",台灣大哥大,http,169244,[12:JkHXq/IsheY1awh5UP+/4D7eh2x7AGGH46ZY1Zu8m:...,"{'12': ['H', 'Xq', '7', 'm', ':', 'a'], '6': [...",[12:+6WwQA56ty1WHXq/Ish+j51h1JcZMsA86nkx7A268H...,"[1578240969.426093, 1578240969.426262, 1578240...","[[1578240969.426093, 1578240969.426262, 157824..."
939,20200106_1700_http,"[1578239431.472793, 1578239434.482427, 1578239...","[Canada, Colombia, United States, Canada, Cana...","[172.105.4.227, 181.129.123.202, 216.244.66.24...",[12:rHXq6x7YNDIsqCjh8wHhFMsml6n61HCY3NkZfsv8m:...,"[(43.6547, -79.3623), (3.5861, -76.5011), (47....",台灣大哥大,http,93561,[12:rHXq6x7YNDIsqCjh8wHhFMsml6n61HCY3NkZfsv8m:...,"{'12': ['HX', 'I', 's', 'w', 'h', ':'], '24': ...",[12:4NBmHXq6x7YNHk/IsqCjh8wHhFMsml6n61HCY3NkZf...,"[1578240050.414508, 1578240050.41473, 15782400...","[[1578240050.414508, 1578240050.41473, 1578240..."
940,20200106_249108104923_http,"[1578289690.645383, 1578302389.776345, 1578338...","[United States, China, China, Morocco, China, ...","[154.202.13.17, 111.230.23.22, 49.234.96.87, 1...",[12:7KS/cHnMQ7s95HfSBBBBBBBBBBBBBSjmHXq/IYDr+b...,"[(37.751, -97.822), (34.7725, 113.7266), (34.7...",台灣大哥大,http,57702,[12:7KS/cHnMQ7s95HfSBBBBBBBBBBBBBSjmHXq/IYDr+b...,"{'12': ['HXq/I', 'x', '7', 'H46Z', ':', 'a', '...",[12:/DfXEmHXq/Is0Kj51h1FMsA86nXx70H46ZY1Zu8m:Y...,"[1578302389.776345, 1578302392.775865, 1578302...","[[1578302389.776345, 1578302392.775865, 157830..."
941,20200106_1440_http,"[1578239428.028117, 1578239433.566138, 1578239...","[Russia, United States, Kazakhstan, Belarus, U...","[46.146.126.112, 66.249.71.108, 85.29.166.247,...",[24:tAzfGCU6BRCTS18YH+w18YHYR385Hltb17KAzfGCU6...,"[(58.0033, 56.2478), (37.751, -97.822), (51.18...",台灣大哥大,http,34045,[24:tAzfGCU6BRCTS18YH+w18YHYR385Hltb17KAzfGCU6...,{'24': ['Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafzno...,[24:tAzfGCU6BRCTS18YH+w18YHYR385Hltb17KAzfGCU6...,"[1578309649.54186, 1578309649.542117, 15783096...","[[1578309649.54186, 1578309649.542117, 1578309..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026,20200107_326417546461_http,"[1578325831.353804, 1578405352.752084]","[Iran, Brazil]","[37.19.88.161, 189.1.185.61]",[6:rHKAqEtEkzBd/AiTeXSj4Hwz5SoW0wfYi7cmDxgoBYX...,"[(35.698, 51.4115), (-22.6097, -47.3823)]",台灣大哥大,http,2,[6:rHKAqEtEkzBd/AiTeXSj4Hwz5SoW0wfYi7cmDxgoBYX...,"{'6': ['A', 'E', 'E', 'e', 'W', 'Dxgo', 'XTECK...",[6:rHKAnDeyARDUWVVnwAR7Gf4OB9EVEeDWTUxAuDxgoP+...,"[1578325831.353804, 1578325831.354201, 1578325...",False
1027,20200111_163208766650_http,"[1578748048.609378, 1578748048.624227]","[Turkey, Turkey]","[88.233.91.179, 88.233.91.179]",[96:bnIRfLr1BglCLXi5XipSNqn32nSMJWsvmkWf37wUGT...,"[(41.0565, 28.9333), (41.0565, 28.9333)]",台灣大哥大,http,2,[96:bnIRfLr1BglCLXi5XipSNqn32nSMJWsvmkWf37wUGT...,"{'96': ['W', 'v', 'm', 'W', '+', 'c', '7', 'hG...",[96:bnIRfLr1BglCLXi5XipSNqn32nSMJWsvmkWf37wUGT...,"[1578748048.609378, 1578748048.609716, 1578748...",False
1028,20200111_231928263111_http,"[1578748120.827261, 1578748133.82611]","[Singapore, Singapore]","[148.66.157.162, 148.66.157.162]",[12:tAMd63FYKnK

In [456]:
pd.DataFrame(df_tmp['packet_timeli'])

,packet_timeli
937,"[[1578262953.089617, 1578262953.089792, 157826..."
938,"[[1578240969.426093, 1578240969.426262, 157824..."
939,"[[1578240050.414508, 1578240050.41473, 1578240..."
940,"[[1578302389.776345, 1578302392.775865, 157830..."
941,"[[1578309649.54186, 1578309649.542117, 1578309..."
...,...
1026,False
1027,False
1028,False
1029,False


In [436]:
dir_path = "/mnt/Raid160TB/pcap_inter/"+str(date[:4])+'_'+str(date[4:6])+'_'+str(date[6:])+"/"+isp+"/intermeidate_data/"
df2,pickle_dir,picture_dir = pickle.load(open(dir_path+"df2_tuples_"+isp+".pkl",'rb'))
df2

,session_time,session_time_list,session_duration,session_i_tt_packet,session_o_tt_packet,session_i_tt_frame_length,session_o_tt_frame_length,udp_i_tt_length,udp_o_tt_length,udp_i_avg_length,...,ip_dst,tcp_srcport,tcp_dstport,country,isp,domain,frame_i_max_protocols,frame_o_max_protocols,tcp_i_payload_list,tcp_o_payload_list
0,1.578285e+09,"[1578285019.2355, 1578285019.235694, 157828501...",0.521155,8,7,899,766,NaN,NaN,NaN,...,175.96.14.82,60795,445,Thailand,None,totinternet.net,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[[6:iCV00dYMe9hNV9oCV/PiCToa0VfCy/a9evgfey9FV1...
1,1.578295e+09,"[1578294672.452107, 1578294672.452335, 1578294...",31.949450,5,4,473,329,NaN,NaN,NaN,...,175.96.15.72,10748,445,Thailand,None,totinternet.net,eth:ethertype:ip:tcp:nbss:smb:gss-api:spnego:n...,eth:ethertype:ip:tcp:nbss:smb,[[3:11Xh03vV1V1Ud2bfV1V1V1V1V1V1V1V1V1V1V133F9...,[[3:11QQoW503vV1V1cdufV1V1V1V1V1V1V1V1V1V1V133...
2,1.578288e+09,"[1578287919.13026, 1578287919.130601, 15782879...",0.213314,8,9,899,886,NaN,NaN,NaN,...,175.96.15.95,53979,445,Taiwan,None,hinet.net,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[[6:iCV00dYMe9hNV9oTEQpQdoa0VfCy/a9evgfey9FV1R...
3,1.578279e+09,"[1578279038.831549, 1578279038.831737, 1578279...",1.287589,8,11,899,1006,NaN,NaN,NaN,...,175.96.48.113,51018,445,Taiwan,None,hinet.net,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[[6:iCV00dYMe9hNV9oM2gfCdoa0VfCy/a9evgfey9FV1R...
4,1.578278e+09,"[1578278264.489654, 1578278264.489803, 1578278...",0.760179,8,10,899,946,NaN,NaN,NaN,...,175.96.48.98,57552,445,Taiwan,None,hinet.net,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[[6:iCV00dYMe9hNV9oM2gfCdoa0VfCy/a9evgfey9FV1R...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850403549373,1.578245e+09,"[1578244996.839178, 1578244996.839447, 1578244...",5.654832,6,3,712,254,NaN,NaN,NaN,...,175.96.49.96,61257,1433,Russia,None,None,eth:ethertype:ip:tcp:tds:gss-api,eth:ethertype:ip:tcp:tds,[[12:McKSxTvuw36WsVBygjifYpVxVZeG2hXiBN1acB4J:...,[[6:Sf/U/93+uST5O3S3QWi2VfaSCI9uMQy6sd:s/U/B+u...
850403549374,1.578271e+09,"[1578270759.481834, 1578270759.48204, 15782707...",9.473710,10,15,1085,1312,NaN,NaN,NaN,...,175.96.49.121,3108,445,Ukraine,None,None,eth:ethertype:ip:tcp:nbss:smb,eth:ethertype:ip:tcp:nbss:smb,[[6:wV08dY0TAqRARUbESX1qvasTgb+RpoedfFKXqEPdcV...,[[6:iCV00dYMe9hNV9oAX5ERoa0VfCy/a9evgfey9FV1RG...
850403549375,1.578274e+09,"[1578274252.166231, 1578274252.166497, 1578274...",3.629581,12,15,1545,2359,NaN,NaN,NaN,...,175.96.49.250,40738,22,Netherlands,None,None,eth:ethertype:ip:tcp:ssh,eth:ethertype:ip:tcp:ssh,"[[3:DfQMPmfK7PATaupm:DPQKtupm, 1578274252.7118...",[[3:DfQMPmflfTAfQW4Y7wUY6ljrA6EkvA6HfQNun:DPQl...
850403549376,1.578315e+09,"[1578315442.691769, 1578315442.691992, 1578315...",3.414751,14,15,2731,2443,NaN,NaN,NaN,...,175.96.14.234,51738,22,United States,None,charter.com,eth:ethertype:ip:tcp:ssh,eth:ethertype:ip:tcp:ssh,[[3:DfQMPmflfTAfQW4Y7wVfV+PGSW+0fYfTQ2fPTRfPjT...,[[3:DfQMPmflfTAfQW4Y7wUY6ljrA6EkvA6HfQNun:DPQl...


In [1]:
df2.iloc[0]

NameError: name 'df2' is not defined

In [445]:
df2.iloc[0,0]

1578285019.2355

In [448]:
tmp = df2[(df2.session_time == 1578314626.453001)|(df2.session_time == 1578285019.2355)]
tmp.session_time_list.tolist()

[[1578285019.2355,
  1578285019.235694,
  1578285019.304065,
  1578285019.304355,
  1578285019.304496,
  1578285019.336862,
  1578285019.423288,
  1578285019.435494,
  1578285019.504771,
  1578285019.513924,
  1578285019.582206,
  1578285019.591076,
  1578285019.684966,
  1578285019.685283,
  1578285019.756655],
 [1578314626.453001,
  1578314626.453227,
  1578314626.752031,
  1578314635.058767,
  1578314635.058937,
  1578314635.093763,
  1578314635.498044,
  1578314641.526114,
  1578314641.563689,
  1578314641.636342,
  1578314641.940879,
  1578314641.945131,
  1578314642.243824]]

In [415]:
# temp = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/亞太電信/case_pickles/http_clusterKey_timelist_paths_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl'
#                        ,'rb'))
# temp

# glob.glob('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/亞太電信/case_pickles/timelist_http*51539658953.pkl')

# temp2 = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/timelist_http_large#193_clusterID#103079217409.pkl'
#                         ,'rb'))
# temp2

# date = '20200106'
# isp = '台灣大哥大'
# proto = 'http'
# pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(date[:4])+'_'+str(date[4:6])+'_'+str(date[6:])+'/'+isp+'/case_pickles/'

# try:
#     (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
#      proto_upgma_dict,stat_df) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
# except ValueError:
#     try:
#         (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
#          proto_upgma_dict,stat_df,df2) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
#     except ValueError:
#         try:
#             (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
#                                     proto_upgma_dict,stat_df) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))
#         except ValueError:
#             (proto_df, proto_df_payload,proto_big_dict,proto_loners,proto_score,proto_cluster_score_dict,
#                                     proto_upgma_dict) = pickle.load(open(pickle_dir+str(date)+'_'+str(proto)+'_all.pkl','rb'))

# o = (1,2,4)
# o[0]

# proto_df.loc[23217,'session_time']

# tmp = agg_proto_df['http']
# 1578262953.089617 in tmp[tmp.isp == '台灣大哥大'].loc[937,'timestamp']

# tmp = pickle.load(open('/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/http_trend_df_20200106_20200107_20200108_20200109_20200110_20200111_20200112.pkl'
#                       ,'rb'))
# tmp

['/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/亞太電信/case_pickles/timelist_http_large#61_clusterID#103079263261.pkl',
 '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/亞太電信/case_pickles/timelist_http_large#15_clusterID#10410.pkl',
 '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/亞太電信/case_pickles/timelist_http_large#70_clusterID#111669215012.pkl',
 '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/亞太電信/case_pickles/timelist_http_large#7_clusterID#1182.pkl',
 '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/亞太電信/case_pickles/timelist_http_large#11_clusterID#1208.pkl',
 '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/亞太電信/case_pickles/timelist_http_large#25_clusterID#1213.pkl',
 '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/亞太電信/case_pickles/timelist_http_large#17_clusterID#12831.pkl',
 '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/亞太電信/case_pickles/timelist_http_large#1_clusterID#13656.pkl',
 '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/亞太電信/case_pickles/timelist_http_large#20_clust

In [343]:
# # from operator import itemgetter 
# # hash_li = ["10","11","12","13","12","15"]
# # taken_idx = [3]
# # i=1
# # compare_idx = [x+i+1 for x in range(len(hash_li)-i-1)]
# # compare_idx = list(set(compare_idx) - set(taken_idx))
# # compare_idx
# # hashes = list(itemgetter(*compare_idx)(hash_li))
# # test = pd.DataFrame(hashes,index=compare_idx)
# # # list(test.index)
# # test

# # li = []
# # li[1]

# kk = '24:Bb/XuYDS02xIQQKJDJOyn/4PVL85J1KZafznoGOuS:sYEHSywVLgu1uS'.split(":")[1:]
# kkk = " : ".join(kk)
# kkk.split(" ")
# # kk

# ssdeep.compare("24:DvS18YH+w18YHYR/XESNEytFa+MVXmdMVX+CYx:DK18hw18z9JtFxUOUOx",
#               "24:tAMuVHDBcXoadjKgQp2hMzPjeZfHKafCqfEKSAMuVHD5C/:+M29Va98khMzCqqMCM2U")

# proto_df.ssdeep_clean.iloc[-2]

# proto_df.ssdeep_flat.iloc[-1]

# tmp = proto_df.loc[1206,'ssdeep_flat']
# for hash_ in tmp:
#     sc = ssdeep.compare('96:vWpyvotA0hvTmAF/vWI+3Dc+vVz4OBWEvAF7vAAP7ay8pshG8p7rwka8/eOe+rsm:vWpyvotA0vTmAF/vWI+3Dc+vVz4OBWEe',
#                   hash_)
#     print(sc)

# ssdeep.compare('96:bnIRfLr1BglCLXi5XipSNqn32nSMJWsvmkWf37wUGT+cZM7hGmhDmEWpEvoIA0+:bnYfLr1alCLXxpSNaZMJWsvmkWfrwUGr',
#               '96:vWpyvotA0hvTmAF/vWI+3Dc+vVz4OBWEvAF7vAAP7ay8pshG8p7rwka8/eOe+rsm:vWpyvotA0vTmAF/vWI+3Dc+vVz4OBWEe')

# from Bio import pairwise2
# from Bio.pairwise2 import format_alignment
# alignments = pairwise2.align.localxx("24:tAt2UIEma7MVXg7gK21JCTS18YH+w18YHWcNgf18YHxK1dqgrFxtCBtDhLcGofA+:+thma7U+O918hw18VcM18xdnXtctdLJq",
#               "24:HMMVC:U")
# print(format_alignment(*alignments[0]))
# li = format_alignment(*alignments[0]).split('\n')
# # alignments
# li

# need_word = []
# for one,two in zip(li[0],li[1]):
#     if two=='|':
#         need_word.append(one)
# "".join(need_word) #中間多次比較只要這樣

# li[3].split("=")[-1]

# alignments = pairwise2.align.localxx(text1,
#                                       "96:sCJkooYBI9iyS1uw1sA75WCJkooYBI9iyQ5Kx:Jy8B9r753y8B995Kx")
# print(format_alignment(*alignments[0]))
# li = format_alignment(*alignments[0]).split('\n')
# need_word = []
# for i,(one,two) in enumerate(zip(li[0],li[1])):
#     if two=='|':
#         need_word.append(one)
#         try:
#             if li[1][i+1]==' ':
#                 need_word.append(" ")
#         except IndexError:
#             break
# kk = "".join(need_word)
# kk.split(" ") #最後才要

# kk

# ssdeep.compare('96:sCJkooYBI9iyS1uw1sA75WCJkooYBI9iyQ5Kx:Jy8B9r753y8B995Kx',
#                '96:7+y28P2BI9XyS1uw1sO5Yy28P2BI9Xyn5Kx:7+yX2BmN5YyX2Bmg5Kx')

# from difflib import SequenceMatcher
# from difflib import context_diffext_diff
# a = "dsa jld lal"
# b = "dsajld kll"
# c = "dsc jle kal"
# d = "dsd jlekal"
# s1 = ['bacon\n', 'eggs\n', 'ham\n', 'guido\n']
# s2 = ['python\n', 'eggy\n', 'hamster\n', 'guido\n']
# context_diff(s1, s2)

# # ss = [a,b,c,d]

# # s = SequenceMatcher()

# # for i in range(len(ss)):
# #     x = ss[i]
# #     s.set_seq1(x)
# #     for j in range(i+1,len(ss)):

# #         y = ss[j]
# #         s.set_seq2(y)

# # #         print
# #         print (s.ratio())
# #         print (s.get_matching_blocks())



# all_df[all_df.index.isin(combine_dict[0])]

# kk['score'].max()

# sorted(combine_dict[0])

# # key_hash_df[[2,3]] = pd.DataFrame(key_hash_df[1].values.tolist(), index=key_hash_df)
# # key_hash_df[[2,3]] = key_hash_df[1].str.split(expand=True)
# # tmp = key_hash_df[1].apply(pd.Series)
# # pd.merge(key_hash_df,tmp,left_index=True, right_index=True)
# # key_hash_df
# # kk1 = tmp.groupby([0]).count()
# # kk2 = tmp.groupby([0]).mean()
# # kk = pd.merge(kk1,kk2,left_index=True, right_index=True)
# # kk['score1'] = kk['1_x']/kk['1_x'].sum()*100
# # kk['score'] = (kk['1_y']+kk['score1'])/2
# # kk['score'].idxmax()
# # print(kk1)
# # kk2
# # kk
# li = ['20200106_295_http','20200106_2984_http']
# kk = need_df.groupby(need_df.idx.isin(li)).agg(list)
# kk = kk[kk.index==True] 
# kk = kk.reset_index(drop=True)
# kk

# kkk = need_df[need_df.idx == '20200106_13656_http']


# yy = {1:[2,3,4],2:[5,6,7]}
# sum(list(yy.values()),[])

# ssdeep.compare('3:OXXUD8EU1V1V1V1RYP/LfIQfdplr+Mfjfun:O0DVPxdL+Mfjmn','3:OXXXL1Vf1V1V1V1RYP/LfIQfdplr+M5DcEYVV91v/vV1V1VV:OQPxdL+M5Avf9')

# ssdeep.compare('48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691uw1Gp1sdnXiftD+','48:+Ka7UzEO918hw18VcM18xdnXtctdLJCAY6S:+do91uw1Gp1sdnXiftD+')

# need_df.loc[0,'ssdeep']

# need_df[need_df.session_num ==need_df.session_num.max()]

# 20200106_20200112_clusterID_time_country_ip_ssdeep.pkl
# 20200106_20200112_clusterID_time_country_ip_ssdeep.pkl

# 2020_01_06/中嘉寬頻/case_pickles/20200106_20200112

# 20200111_429496734683_http

,0
2,12
4,12
5,15


# SWEMA (比較趨勢)
1. 先將不同isp合併被認為是高度相似的clusters，減少數量提高效率增加準確性
2. 拿出全部的cluster名稱做為col
3. 將每個cluster所包含的全部session time換成日期做為row
4. 計算每個日期各個cluster所包含的session數量
    
    4-1. 利用softmax將每個日期的session數量轉換為和為1的向量 (sum=1 & 0<=value<=1)
    
    4-2. 把每個日期轉換為one-hot vector
    
    4-3. 統計各天的全部session數量
    
    4-4. 統計各天所包含手法cluster的數量
5. 計算差一天-差七天的EMA(N=8-2)

    5-1. 分子為不同N所得EMA所得的4-1.cosine similarity、4-2.jaccard similarity、4-3.差異變化量(看量有幾%是一樣的)、4-4.差異變化量(看量有幾%是一樣的)
    
    5-2. 分母為不同N EMA之alpha分母和
    
6. 給予4.不同的加權值，計算加權平均後得到SWEMA分數

    6-1. 某天分數越小，代表這一天相較於所有其他前面天的變化是大的
    
    6-2. 某天分數相較於其他天變化劇烈，代表這一天跟其他天有異常的情況發生

In [7]:
proto_li =  ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip'] #改!!'tds',
# date_li2 = ["0106","0107","0108","0109","0110","0111","0112"] 
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"]  #改!!
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','遠傳電信','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
# date_li = ['0102','0130','0110']
file_name = "_".join(sorted(date_li)) #改!!
min_date = str(min(date_li))
max_date = str(max(date_li))

In [10]:
pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/台灣大哥大/case_pickles/crossisp_signature_dfdict.pkl' #改
agg_proto_df=pickle.load(open(pickle_dir,'rb'))
# pickle.dump(obj=agg_proto_df,file=open(pickle_dir+'crossisp_ssdeep_dfdict.pkl','wb'))
agg_proto_df['http']

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,signature,ssdeep_clean,clusterkey_timelist
0,20200106_13656_http,"[1578239437.622562, 1578239439.00718, 15782394...","[United States, United States, United States, ...","[66.249.65.188, 66.249.79.137, 66.249.71.43, 6...",[48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691uw...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",亞太電信,http,183041,[48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691uw...,"{'12': ['HXq', '6xmo/+R', 'Nk', 'g', 'c', 'G',...",[12:7XiS9HXq6xmo/+R+pHCFNkDTg1IEcuzGpfAvTYXNkZ...,"[1578270108.689238, 1578270108.68945, 15782701..."
1,20200106_295_http,"[1578239440.513196, 1578239443.592105, 1578239...","[Iran, Russia, United States, United States, R...","[79.132.201.185, 212.41.14.170, 216.244.66.245...",[12:rHXq6xmMVkExShI/XES+AoGY1uqh5U9Z1J3vhm:Da7...,"[(35.698, 51.4115), (56.003, 92.7993), (47.722...",亞太電信,http,81808,[12:rHXq6xmMVkExShI/XES+AoGY1uqh5U9Z1J3vhm:Da7...,"{'12': ['HX', 'x', 'I', '1', 'm:'], '24': ['n'...",[12:rHXAxmMVXo5wVfHClIshpeR56Y1awh5UPifVsy+m:D...,"[1578268769.767172, 1578268769.767411, 1578268..."
2,20200106_2984_http,"[1578239434.537772, 1578239437.816006, 1578239...","[Russia, None, United States, United States, U...","[5.188.84.45, 109.70.100.27, 66.249.68.16, 66....",[24:tAxtAKAFMVvoCYEQTcZMzP2tS18YH+w18YHxKt1Hqa...,"[(55.7386, 37.6068), (None, None), (37.751, -9...",亞太電信,http,64609,[24:tAxtAKAFMVvoCYEQTcZMzP2tS18YH+w18YHxKt1Hqa...,"{'24': ['+', 'w', '8', 'H', 'Y', ':', 'U'], '3...",[24:9SAvS18YH+w18YHYR385HltF0Vrde4+MVZMkMVHCYx...,"[1578268911.573342, 1578268911.573646, 1578268..."
3,20200106_51539658953_http,"[1578243129.379452, 1578243147.033231, 1578243...","[China, China, China, China, China, China, Hon...","[103.140.126.152, 103.140.126.152, 103.140.126...",[12:72wXcHnMQ7s95HfSBBBBBBBSjmHXq/Ish+j51h1JcZ...,"[(34.7725, 113.7266), (34.7725, 113.7266), (34...",亞太電信,http,48240,[12:72wXcHnMQ7s95HfSBBBBBBBSjmHXq/Ish+j51h1JcZ...,"{'12': ['HXq', 'x', 'm', '6', 'Z', ':', 'a'], ...",[12:+KmHXq/Ish+j51h1sXxMsA86njXAxmMVkGH46ZY1Zu...,"[1578305243.631025, 1578305243.631348, 1578305..."
4,20200106_152_http,"[1578239434.511962, 1578239437.425728, 1578239...","[China, China, China, China, China, China, Chi...","[122.14.208.63, 139.199.74.166, 148.70.163.11,...",[12:/ADXEmHXq/I75cuXNFljI4DxmMVXqH46ZY1Zu8m:zA...,"[(23.1167, 113.25), (34.7725, 113.7266), (34.7...",亞太電信,http,31897,[12:/ADXEmHXq/I75cuXNFljI4DxmMVXqH46ZY1Zu8m:zA...,"{'12': ['7', 'X', 'j', 'm', 'm:', 'M'], '24': ...",[12:/ADXEmHXq/I75cuXNFljI4DxmMVXqH46ZY1Zu8m:zA...,"[1578286833.368446, 1578286833.3688, 157828683..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,"[20200109_317827601065_http, 20200109_16320876...","[[1578545206.609212, 1578545369.126079, 157854...","[[Russia, Russia, Russia, Russia, Russia, Russ...","[[193.23.3.36, 193.23.3.36, 193.23.3.36, 193.2...",[[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA...,"[[(55.7386, 37.6068), (55.7386, 37.6068), (55....","[台灣碩網, 台灣碩網]","[http, http]","[9, 2]",[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA4...,"{'48': ['a', 'Q5jqXMz', 'A', 'q', ':', '5Rz', ...",[48:aAaeQ5jqXMziAfdGqUGqXZGqXZGqdKGqtl:u5RziA4...,"[[1578545206.609212, 1578545206.609449, 157854..."
1203,"[20200107_206158449329_http, 20200106_12025908...","[[1578333098.110814, 1578398130.044447, 157860...","[[Germany, Russia, United States, Ukraine, Ukr...","[[185.220.101.79, 5.188.210.65, 207.97.174.134...",[[96:Cdo2Hqv5BI93yS1uw1Acz5W5BI93yx5Kx:Cdo2Kv5...,"[[(51.2993, 9.491), (55.7386, 37.6068), (40.71...","[台灣碩網, 台灣碩網, 台灣碩網]","[http, http, http]","[5, 3, 3]",[96:Cdo2Hqv5BI93yS1uw1Acz5W5BI93yx5Kx:Cdo2Kv5B...,"{'96': ['B', 'I', '9', 'yS1uw1', 'B', 'I', '9'...",[96:7+y28P2BI9XyS1uw1sO5Yy28P2BI9Xyn5Kx:7+yX2B...,"[[1578333098.110814, 1578333098.111041, 157833..."
1204,"[20200109_85899367448_http, 20200109_858994174...","[[1578545199.046963, 1578565438.876521, 157856...","[[Russia, Russia, Russia], [Russia, Russia]]","[[193.23.3.36, 193.23.3.36, 193.

In [8]:
# proto = proto_li[2]
# isp = isp_li[-1]
# temp = pickle.load(file=open('/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'+proto+'_trend_df_'+file_name+'.pkl','rb')) #改!!
# temp

# temp.iloc[0,1][-1]

# from datetime import datetime
# timestamp = 1578757564.275287
# dt_object = datetime.fromtimestamp(timestamp)
# dt_object

# str(dt_object).split(" ")[0]

# ftp_idxli = agg_proto_df['ftp'].idx.tolist()
# flat_idx = []
# for liostr in ftp_idxli:
#     if type(liostr)==list:
#         flat_idx.extend(liostr)
#     else:
#         flat_idx.append(liostr)
# # kk = list(functools.reduce(operator.iconcat, ftp_idxli, []))
# flat_idx

# "20200107_51539626516_ftp" in flat_idx

,idx,timestamp,country,src_ip,ssdeep
0,20200106_789,"[1578244607.082916, 1578254060.997759, 1578254...","[United States, China, China, China, China, Ch...","[108.176.66.39, 123.207.160.243, 123.207.160.2...","[3:Af5R9A:AhR9A, 3:A5mFfy7RJcE/qrm:A5QfyVSoqS,..."
1,20200107_305,"[1578325961.102387, 1578325979.148913, 1578325...","[China, China, China, China, China, Turkey, Ch...","[123.207.160.243, 123.207.160.243, 123.207.160...","[3:A5mFfdtjSfAAFEn:A5QfG4AFEn, 3:A5mFfuv3WYLun..."
2,20200106_788,"[1578239428.2957, 1578239469.938955, 157823955...","[None, France, France, France, France, France,...","[35.241.250.32, 94.23.13.194, 51.255.70.132, 9...","[3:FfDdNTGCij9En:lryCij9En, 3:A5mFfuWYW+QptfWc..."
3,20200106_1585,"[1578239426.259187, 1578239569.911595, 1578239...","[United States, United States, United States, ...","[170.130.187.42, 104.140.188.22, 170.130.187.3...","[3:2f0/9B9En:2f0/j9En, 3:2f0/9B9En:2f0/j9En, 3..."
4,20200109_255,"[1578498806.823722, 1578498823.611793, 1578498...","[Iran, Iran, Iran, Iran, Iran, Iran, Iran, Ira...","[89.42.209.111, 89.42.209.111, 89.42.209.111, ...","[3:A5mFfKS2Sd9fhfuROWPWYLun:A5QfES9gOKWeun, 3:..."
5,20200106_795,"[1578240973.893346, 1578241423.049947, 1578253...","[Japan, Japan, Japan, Japan, Japan, Japan, Rus...","[172.105.218.213, 172.105.218.213, 172.105.218...",[3:OXXXL1Vf1V1V1V1RYP/LfIQfdplr+M5DcEYVV91v/2v...
6,20200106_456,"[1578239441.46043, 1578239471.963299, 15782394...","[Romania, Romania, Romania, Romania, Romania, ...","[83.97.20.46, 83.97.20.46, 83.97.20.46, 83.97....","[3:Sfrfun:Sfrmn, 3:Sfrfun:Sfrmn, 3:Sfrfun:Sfrm..."
7,20200106_2359,"[1578239449.098435, 1578239449.254335, 1578239...","[Romania, Romania, Romania, Romania, Romania, ...","[83.97.20.46, 83.97.20.46, 83.97.20.46, 83.97....","[3:aRLu:atu, 3:aRLu:atu, 3:aRLu:atu, 3:aRLu:at..."
8,20200106_25769804548,"[1578253835.958314, 1578253844.415693, 1578254...","[China, China, China, China, China, China, Chi...","[123.207.160.243, 123.207.160.243, 123.207.160...","[3:FfDdNS+6S6XfTb8LVE:lrS+6S6PCu, 3:FfDdNS+6S6..."
9,20200106_790,"[1578258918.04327, 1578258927.338335, 15782589...","[China, China, China, China, China, China, Chi...","[106.13.39.196, 106.13.39.196, 106.13.39.196, ...","[3:FfDdNSJv3WYLun:lrSBWeun, 3:FfDdNSJvqm:lrS7,..."


In [33]:
# for i,isp in enumerate(isp_li):
#     pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'
#     try:
#         isp_all_df = pickle.load(file=open(pickle_dir+'clusterName_overview_denoise_df_'+str(min_date)+'_'+str(date_li[-1])+'.pkl','rb'))
#     except FileNotFoundError:
#         print("!!!File not Found:",pickle_dir+'clusterName_overview_denoise_df_'+str(min_date)+'_'+str(date_li[-1])+'.pkl!!!')
#         continue
#     isp_all_df['idx'] = isp_all_df['idx']+'_'+isp
#     if i == 0:
#         all_df = isp_all_df
#     else:
#         all_df = all_df.append(isp_all_df)
# all_df = all_df.reset_index(drop=True)
# all_df

!!!File not Found: /home/antslab/NAS1_RAID6/pcap_inter/2020_01_06/遠傳電信/case_pickles/clusterName_overview_denoise_df_20200106_20200112.pkl!!!


,idx,timestamp,country,src_ip,ssdeep
0,20200106_78_http_中嘉寬頻,"[1578239432.58498, 1578239433.992329, 15782394...","[South Korea, United States, United States, Br...","[14.33.65.179, 216.244.66.231, 216.244.66.240,...",[12:rHXq6xydG25Is0+R56Y1awh5UPQVJJtzpvhm:DahdN...
1,20200106_101_http_中嘉寬頻,"[1578239454.984635, 1578239456.281346, 1578239...","[Colombia, Russia, China, United States, Unite...","[200.25.254.148, 176.123.164.240, 159.138.155....",[96:7HcnXFUpSBMHsUpSBwHV3pxnXFQjek3FI6nXFF1OFT...
2,20200106_20616_http_中嘉寬頻,"[1578242853.096138, 1578242855.813803, 1578242...","[China, China, China, China, China, China, Chi...","[132.232.106.82, 132.232.106.82, 132.232.106.8...",[24:E9Bkwa2/Gm4e04m3/fspiWxJu3palUTQQMc2ukQQ5H...
3,20200106_17318_http_中嘉寬頻,"[1578239436.832176, 1578239452.813811, 1578239...","[United States, United States, United States, ...","[66.249.68.17, 66.249.66.83, 66.249.79.154, 66...",[48:ejVMaPO918hw18VcM18xdnXtctdLJCAY6S:j91uw1G...
4,20200108_153_http_中嘉寬頻,"[1578412231.033238, 1578412232.22328, 15784122...","[Japan, Japan, Hong Kong, Japan, Japan, Japan,...","[137.220.184.55, 137.220.184.55, 39.109.105.25...",[96:LQ0eZOnXNLQ0zZCPLQ0JZnPUVYZ8nXNUVBZcPUVbZ1...
...,...,...,...,...,...
6738,20200112_274877915564_telnet_台灣碩網,"[1578757921.023994, 1578757926.139868]","[Germany, Germany]","[185.216.213.179, 185.216.213.179]","[3:DfWrTmfzYVE:DQmf0VE, 3:2feon:2fzn, 3:2feon:..."
6739,20200108_3875_sip_台灣碩網,"[1578432989.625082, 1578432989.625162, 1578432...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[24:pv7mDETJ1tHPW8gJR0YfYJPQKiR+b3Za7gcXYEXl8X...
6740,20200106_171798691856_sip_台灣碩網,"[1578242202.876367, 1578242244.23576, 15782422...","[United States, United States, United States, ...","[104.140.188.6, 104.206.128.54, 104.206.128.78...",[24:qgOf3vI7mDL5gizTKiqf3vX4v/DRt0UUgbYMfy1wUX...
6741,20200109_4186_sip_台灣碩網,"[1578573525.641612, 1578573525.641668, 1578573...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[48:maadDvJfeaI08oKY18Vje821BCkOPk/Ix:VcDvJfeR...


In [41]:

date_proto_df = {}

def count_date_sessions(liorlioli):
    '''
    GOAL: construct a dict, key=date, value=session#
    Input: list or list of list (element is float)
    Output: dict
    '''
    date_dict = {}
    if type(liorlioli[0]) == list: #li of li
        for liotime in liorlioli:
            for timestamp in liotime:
                dt_object = datetime.fromtimestamp(timestamp)
                date_str = str(dt_object).split(" ")[0]
                date_str = date_str.replace('-','')
                try:
                    tmp = date_dict[date_str]
                    tmp = tmp + 1
                    date_dict[date_str] = tmp
                except KeyError:
                    date_dict[date_str] = 1
    else: #li of float
        for timestamp in liorlioli:
            dt_object = datetime.fromtimestamp(timestamp)
            date_str = str(dt_object).split(" ")[0]
            date_str = date_str.replace('-','')
            try:
                tmp = date_dict[date_str]
                tmp = tmp + 1
                date_dict[date_str] = tmp
            except KeyError:
                date_dict[date_str] = 1        
    return date_dict

def chg_idx_name(stroli):
    if type(stroli)==str:
        return stroli
    else:
        return "_".join(stroli)

for proto in tqdm(proto_li):
    proto_df = agg_proto_df[proto]
    proto_df['date_count'] = proto_df.timestamp.map(count_date_sessions)
    proto_df['idx'] = proto_df.idx.map(chg_idx_name)
    date_proto_df[proto] = proto_df
#     break
proto_df

100%|██████████| 11/11 [03:13<00:00, 17.55s/it]


,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,ssdeep_clean,signature,clusterkey_timelist,date_count
0,20200106_14243_sip,"[1578239440.238951, 1578239443.45521, 15782394...","[United States, United States, United States, ...","[104.140.188.2, 104.206.128.46, 104.206.128.22...",[12:qYlim7mDLRfAaAz85g9qX6oa+sCssVWGY07GiDv+5g...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",亞太電信,sip,1146,[12:qYlim7mDLRfAaAz85g9qX6oa+sCssVWGY07GiDv+5g...,[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...,"{'12': ['A', 'z', '5g9qX6oa+sCssVWGY07GiDv+5gT...","[1578293229.060841, 1578293229.061113, 1578293...","{'20200105': 15, '20200106': 277, '20200107': ..."
1,20200106_77309423428_sip,"[1578320613.049779, 1578320613.105093, 1578320...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[24:pSAc1hnz09Ac6AcTUnQmDED8XUi1xbe/jv8XUi1EBZ...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",亞太電信,sip,324,[24:pSAc1hnz09Ac6AcTUnQmDED8XUi1xbe/jv8XUi1EBZ...,[24:pP+Jchnz0IU+JMU+JpxumDED8ePgguc8e6BZsKiXYE...,"{'24': ['mDE', 'Ki', 'XYEXl', 'Q', ':', 'h', '...","[1578320613.105093, 1578320613.105328, 1578320...","{'20200106': 3, '20200107': 2, '20200110': 11,..."
2,20200106_13013_sip,"[1578314532.079033, 1578314532.114186, 1578314...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[48:0U5La8En3uU5NPU5O39elU08oKY18Vje821BCkOPk/...,"[(48.8582, 2.3387), (48.8582, 2.3387), (48.858...",亞太電信,sip,222,[48:0U5La8En3uU5NPU5O39elU08oKY18Vje821BCkOPk/...,[48:0U5La8En3uU5NPU5O39elU08oKY18Vje821BCkOPk/...,"{'48': ['e', 'U', '08oKY18Vje821BCkOPk/Ix:', '...","[1578314686.818419, 1578314686.818657, 1578314...",{'20200106': 222}
3,20200106_8589948585_sip,"[1578258144.368324, 1578258144.418974, 1578258...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[77.247.110.179, 77.247.110.179, 77.247.110.17...",[24:sAnWPEgsp5k7mDEUd1dgF2EKimAnWPEgspYCSnWPEg...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",亞太電信,sip,7,[24:sAnWPEgsp5k7mDEUd1dgF2EKimAnWPEgspYCSnWPEg...,[24:sAnWPEgsp5k7mDEUd1dgF2EKimAnWPEgspYCSnWPEg...,"{'24': ['Egsp5k7mDEUd1', 'g', 'E', 'KimAn', 'E...","[1578258144.368324, 1578258144.368677, 1578258...",{'20200106': 7}
4,20200106_51539621821_sip,"[1578320874.953381, 1578320874.955226, 1578320...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[24:pSMfChnz09MfXMffnQmDED8Xzg1ge0v8Xz/BZsKiXY...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",中華電信,sip,9430,[24:pSMfChnz09MfXMffnQmDED8Xzg1ge0v8Xz/BZsKiXY...,[24:pSMFChnz09MFXMFfnQmDED8K1PqonSjv8FBZsKiXYE...,"{'24': ['M', 'mDE', 'Y', 'g', '0']}","[1578320875.392048, 1578320875.392294, 1578320...","{'20200106': 10, '20200108': 66, '20200110': 4..."
5,20200106_465_sip,"[1578239559.052485, 1578239573.363544, 1578239...","[United States, United States, United States, ...","[104.140.188.14, 104.140.188.14, 170.130.187.1...",[24:qgOf3vI7mDL5gizTKiqf3vX4v/DRt0UUgbYMfy1wUX...,"[(37.751, -97.822), (37.751, -97.822), (34.054...",中華電信,sip,3476,[24:qgOf3vI7mDL5gizTKiqf3vX4v/DRt0UUgbYMfy1wUX...,[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...,"{'12': ['A', 'z', '5g9qX6oa+sCssVWGY07GiDv+5gT...","[1578240647.718236, 1578240647.718447, 1578240...","{'20200105': 14, '20200106': 612, '20200107': ..."
6,20200106_15702_sip,"[1578258159.385014, 1578258159.400618, 1578258...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[77.247.110.179, 77.247.110.179, 77.247.110.17...",[24:sPLTON5F7mDEUd1TjgMRPwKimPLTON5NC9LTON51ef...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",中華電信,sip,153,[24:sPLTON5F7mDEUd1TjgMRPwKimPLTON5NC9LTON51ef...,[24:sPLTON5F7mDEUd1TjgMRPwKimPLTON5NC9LTON51ef...,"{'24': ['N5F7mDEUd1', 'g', 'KimPL', 'N5NC9L', ...","[1578258172.813086, 1578258175.811413, 1578258...",{'20200106': 153}
7,20200109_43753_sip,"[1578572318.279269, 1578572318.287146, 1578572...","[France, France, France, France, France, Franc...","[5.135.

In [48]:
# def chg_idx_name(stroli):
#     if type(stroli)==str:
#         return stroli
#     else:
#         return "_".join(stroli)
# for proto in tqdm(proto_li):
#     proto_df = agg_proto_df[proto]
#     proto_df['idx'] = proto_df.idx.map(chg_idx_name)
#     date_proto_df[proto] = proto_df
# proto_df

100%|██████████| 11/11 [00:00<00:00, 2694.78it/s]


,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num,ssdeep_flat,ssdeep_clean,signature,clusterkey_timelist,date_count
0,20200106_14243_sip,"[1578239440.238951, 1578239443.45521, 15782394...","[United States, United States, United States, ...","[104.140.188.2, 104.206.128.46, 104.206.128.22...",[12:qYlim7mDLRfAaAz85g9qX6oa+sCssVWGY07GiDv+5g...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",亞太電信,sip,1146,[12:qYlim7mDLRfAaAz85g9qX6oa+sCssVWGY07GiDv+5g...,[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...,"{'12': ['A', 'z', '5g9qX6oa+sCssVWGY07GiDv+5gT...","[1578293229.060841, 1578293229.061113, 1578293...","{'20200105': 15, '20200106': 277, '20200107': ..."
1,20200106_77309423428_sip,"[1578320613.049779, 1578320613.105093, 1578320...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[24:pSAc1hnz09Ac6AcTUnQmDED8XUi1xbe/jv8XUi1EBZ...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",亞太電信,sip,324,[24:pSAc1hnz09Ac6AcTUnQmDED8XUi1xbe/jv8XUi1EBZ...,[24:pP+Jchnz0IU+JMU+JpxumDED8ePgguc8e6BZsKiXYE...,"{'24': ['mDE', 'Ki', 'XYEXl', 'Q', ':', 'h', '...","[1578320613.105093, 1578320613.105328, 1578320...","{'20200106': 3, '20200107': 2, '20200110': 11,..."
2,20200106_13013_sip,"[1578314532.079033, 1578314532.114186, 1578314...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[48:0U5La8En3uU5NPU5O39elU08oKY18Vje821BCkOPk/...,"[(48.8582, 2.3387), (48.8582, 2.3387), (48.858...",亞太電信,sip,222,[48:0U5La8En3uU5NPU5O39elU08oKY18Vje821BCkOPk/...,[48:0U5La8En3uU5NPU5O39elU08oKY18Vje821BCkOPk/...,"{'48': ['e', 'U', '08oKY18Vje821BCkOPk/Ix:', '...","[1578314686.818419, 1578314686.818657, 1578314...",{'20200106': 222}
3,20200106_8589948585_sip,"[1578258144.368324, 1578258144.418974, 1578258...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[77.247.110.179, 77.247.110.179, 77.247.110.17...",[24:sAnWPEgsp5k7mDEUd1dgF2EKimAnWPEgspYCSnWPEg...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",亞太電信,sip,7,[24:sAnWPEgsp5k7mDEUd1dgF2EKimAnWPEgspYCSnWPEg...,[24:sAnWPEgsp5k7mDEUd1dgF2EKimAnWPEgspYCSnWPEg...,"{'24': ['Egsp5k7mDEUd1', 'g', 'E', 'KimAn', 'E...","[1578258144.368324, 1578258144.368677, 1578258...",{'20200106': 7}
4,20200106_51539621821_sip,"[1578320874.953381, 1578320874.955226, 1578320...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[24:pSMfChnz09MfXMffnQmDED8Xzg1ge0v8Xz/BZsKiXY...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",中華電信,sip,9430,[24:pSMfChnz09MfXMffnQmDED8Xzg1ge0v8Xz/BZsKiXY...,[24:pSMFChnz09MFXMFfnQmDED8K1PqonSjv8FBZsKiXYE...,"{'24': ['M', 'mDE', 'Y', 'g', '0']}","[1578320875.392048, 1578320875.392294, 1578320...","{'20200106': 10, '20200108': 66, '20200110': 4..."
5,20200106_465_sip,"[1578239559.052485, 1578239573.363544, 1578239...","[United States, United States, United States, ...","[104.140.188.14, 104.140.188.14, 170.130.187.1...",[24:qgOf3vI7mDL5gizTKiqf3vX4v/DRt0UUgbYMfy1wUX...,"[(37.751, -97.822), (37.751, -97.822), (34.054...",中華電信,sip,3476,[24:qgOf3vI7mDL5gizTKiqf3vX4v/DRt0UUgbYMfy1wUX...,[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...,"{'12': ['A', 'z', '5g9qX6oa+sCssVWGY07GiDv+5gT...","[1578240647.718236, 1578240647.718447, 1578240...","{'20200105': 14, '20200106': 612, '20200107': ..."
6,20200106_15702_sip,"[1578258159.385014, 1578258159.400618, 1578258...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[77.247.110.179, 77.247.110.179, 77.247.110.17...",[24:sPLTON5F7mDEUd1TjgMRPwKimPLTON5NC9LTON51ef...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",中華電信,sip,153,[24:sPLTON5F7mDEUd1TjgMRPwKimPLTON5NC9LTON51ef...,[24:sPLTON5F7mDEUd1TjgMRPwKimPLTON5NC9LTON51ef...,"{'24': ['N5F7mDEUd1', 'g', 'KimPL', 'N5NC9L', ...","[1578258172.813086, 1578258175.811413, 1578258...",{'20200106': 153}
7,20200109_43753_sip,"[1578572318.279269, 1578572318.287146, 1578572...","[France, France, France, France, France, Franc...","[5.135.

In [154]:
#row = date, col = cluster name
# 先建count表，再建ratio跟on-hot的表
freq_proto_df = {} #所有ISP，各protocol各天的次數表
for proto in tqdm(proto_li):
    proto_df = date_proto_df[proto]
    temp = pd.DataFrame(index = date_li,columns=proto_df.idx)
    for i in range(len(proto_df)):
        row = proto_df.iloc[i]
        idx = row['idx']
        dt_dict = row['date_count']
        for k,v in dt_dict.items():
            if k not in date_li:
                continue
            temp.loc[k,idx] = v
    temp = temp.fillna(0)
    freq_proto_df[proto] = temp
pickle.dump(obj=(date_proto_df,freq_proto_df), #改路徑!
            file=open('/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/中華電信/intermeidate_data/crossisp_dateCount_freqTable.pkl','wb'))
temp

100%|██████████| 11/11 [00:02<00:00,  5.46it/s]


idx,20200106_14243_sip,20200106_77309423428_sip,20200106_13013_sip,20200106_8589948585_sip,20200106_51539621821_sip,20200106_465_sip,20200106_15702_sip,20200109_43753_sip,20200110_34359759799_sip,20200111_266287981719_sip,...,20200106_42949704991_sip,20200106_17559_sip,20200106_17467_sip,20200109_2562_sip,20200106_188978602930_sip,20200106_14243_sip_20200106_465_sip_20200106_8452_sip_20200106_8589960531_sip_20200106_1876_sip_20200106_171798691856_sip_20200106_3929_sip_20200106_26461_sip_20200106_1589_sip_20200106_17559_sip,20200106_77309423428_sip_20200106_13013_sip_20200106_51539621821_sip_20200109_43753_sip_20200111_266287981719_sip_20200106_26978_sip_20200110_24448_sip_20200108_94489305705_sip_20200106_10024_sip_20200108_3875_sip_20200109_4186_sip_20200108_25769829215_sip_20200106_8589944070_sip_20200106_17179898116_sip_20200110_688_sip_20200106_13657_sip_20200106_42949704991_sip,20200106_8589948585_sip_20200106_15702_sip_20200106_17467_sip,20200110_34359759799_sip_20200109_32722_sip_20200106_26617_sip_20200109_2562_sip,20200111_1065151919515_sip_20200106_541165880611_sip_20200111_154618826274_sip_20200106_188978602930_sip
20200106,277,3,222,7,10,612,153,0,0,0,...,7,101,64,0,2,2275,1007,224,50,4
20200107,201,2,0,0,0,783,0,0,0,0,...,0,153,0,0,0,2803,2,0,0,0
20200108,116,0,0,0,66,681,0,0,0,0,...,56,102,0,0,5,1775,268,0,0,5
20200109,116,0,0,0,0,341,0,66,0,0,...,68,55,0,60,0,1075,200,0,118,0
20200110,96,11,0,0,4741,422,0,0,14,0,...,1468,2,0,0,0,1214,44093,0,14,0
20200111,175,31,0,0,2808,418,0,0,0,2,...,118,2,0,0,0,1167,15330,0,0,4
20200112,150,277,0,0,1805,205,0,0,0,0,...,10189,29,0,0,2,1575,99164,0,0,2


In [155]:
cos_matrix = {} #建立頻率表
jac_matrix = {} #建立one-hot表
def chg_onehot(num):
    if num < 1e-100:
        return 0
    else:
        return 1
for proto in tqdm(proto_li):
    proto_df = freq_proto_df[proto]
    proto_df2 = proto_df[:]
    proto_df.loc[:,'sum'] = proto_df.sum(axis=1)
#     proto_df.loc["total",:] = proto_df.sum(axis=0)    
    for date in date_li:
        proto_df.loc[date,proto_df.columns[:-1]] = proto_df.loc[date,proto_df.columns[:-1]]/proto_df.loc[date,'sum']
    temp = proto_df.iloc[:,:-1]
    tmp = temp.applymap(chg_onehot)
    cos_matrix[proto] = temp
    jac_matrix[proto] = tmp
#     proto_df2 = freq_proto_df[proto]
    proto_df2.loc[:,'sum'] = proto_df2.sum(axis=1)
    proto_df2.loc[:,'sum_cluster'] = tmp.sum(axis=1)
    proto_df2.loc["total",:] = proto_df2.sum(axis=0)    
#     proto_df2.loc['total','sum_cluster'] = proto_df['sum_cluster'].sum()
    
    freq_proto_df[proto] = proto_df2 #建立統計表
#     break
#     freq_proto_df[proto] = proto_df
pickle.dump(obj=(cos_matrix,jac_matrix,freq_proto_df),
            file=open('/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/中華電信/intermeidate_data/crossisp_simMatrixs_statTable.pkl','wb'))
proto_df2

100%|██████████| 11/11 [00:05<00:00,  2.08it/s]


idx,20200106_14243_sip,20200106_77309423428_sip,20200106_13013_sip,20200106_8589948585_sip,20200106_51539621821_sip,20200106_465_sip,20200106_15702_sip,20200109_43753_sip,20200110_34359759799_sip,20200111_266287981719_sip,...,20200106_17467_sip,20200109_2562_sip,20200106_188978602930_sip,20200106_14243_sip_20200106_465_sip_20200106_8452_sip_20200106_8589960531_sip_20200106_1876_sip_20200106_171798691856_sip_20200106_3929_sip_20200106_26461_sip_20200106_1589_sip_20200106_17559_sip,20200106_77309423428_sip_20200106_13013_sip_20200106_51539621821_sip_20200109_43753_sip_20200111_266287981719_sip_20200106_26978_sip_20200110_24448_sip_20200108_94489305705_sip_20200106_10024_sip_20200108_3875_sip_20200109_4186_sip_20200108_25769829215_sip_20200106_8589944070_sip_20200106_17179898116_sip_20200110_688_sip_20200106_13657_sip_20200106_42949704991_sip,20200106_8589948585_sip_20200106_15702_sip_20200106_17467_sip,20200110_34359759799_sip_20200109_32722_sip_20200106_26617_sip_20200109_2562_sip,20200111_1065151919515_sip_20200106_541165880611_sip_20200111_154618826274_sip_20200106_188978602930_sip,sum,sum_cluster
20200106,277.0,3.0,222.0,7.0,10.0,612.0,153.0,0.0,0.0,0.0,...,64.0,0.0,2.0,2275.0,1007.0,224.0,50.0,4.0,7120.0,30.0
20200107,201.0,2.0,0.0,0.0,0.0,783.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2803.0,2.0,0.0,0.0,0.0,5610.0,13.0
20200108,116.0,0.0,0.0,0.0,66.0,681.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,1775.0,268.0,0.0,0.0,5.0,4096.0,19.0
20200109,116.0,0.0,0.0,0.0,0.0,341.0,0.0,66.0,0.0,0.0,...,0.0,60.0,0.0,1075.0,200.0,0.0,118.0,0.0,2786.0,17.0
20200110,96.0,11.0,0.0,0.0,4741.0,422.0,0.0,0.0,14.0,0.0,...,0.0,0.0,0.0,1214.0,44093.0,0.0,14.0,0.0,90642.0,24.0
20200111,175.0,31.0,0.0,0.0,2808.0,418.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,1167.0,15330.0,0.0,0.0,4.0,33002.0,24.0
20200112,150.0,277.0,0.0,0.0,1805.0,205.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,1575.0,99164.0,0.0,0.0,2.0,201482.0,23.0
total,1131.0,324.0,222.0,7.0,9430.0,3462.0,153.0,66.0,14.0,2.0,...,64.0,60.0,9.0,11884.0,160064.0,224.0,182.0,15.0,344738.0,150.0


In [182]:
#EMA公式，可以都在修正成for迴圈
alpha1 = 2/(7+1) #隔1天，N=7
alpha2 = 2/(6+1)
alpha3 = 2/(5+1)
alpha4 = 2/(4+1)
alpha5 = 2/(3+1)
alpha6 = 2/(2+1) #隔6天，N=2
proto_score_dict = {}
for proto in tqdm(proto_li):
    score_dict = {}
    cos_proto = cos_matrix[proto]
    jac_proto = jac_matrix[proto]   
    stat_proto = freq_proto_df[proto]
    for i,date in enumerate(date_li):
        if i ==0:
            continue
        score1 = []
        score2 = []
        score3 = []
        score4 = []
        score5 = []
        score6 = []
        up = 0
        down = 0
        for k in range(i):
            sim_score1 = 0.5*cosine_similarity(np.array([cos_proto.iloc[i-k].values]),np.array([cos_proto.iloc[i-1-k].values]))[0][0] + 0.5*jaccard_score(jac_proto.iloc[i-k].values,jac_proto.iloc[i-1-k].values)    
            session_stat = min(stat_proto['sum'].iloc[i-k],stat_proto['sum'].iloc[i-1-k])/max(stat_proto['sum'].iloc[i-k],stat_proto['sum'].iloc[i-1-k])
            cluster_stat = min(stat_proto['sum_cluster'].iloc[i-k],stat_proto['sum_cluster'].iloc[i-1-k])/max(stat_proto['sum_cluster'].iloc[i-k],stat_proto['sum_cluster'].iloc[i-1-k])   
            score1.append((sim_score1,session_stat,cluster_stat)) #存p11,p12,p13,...
        for i,(p,s1,s2) in enumerate(score1):
            up = up + (1-alpha1)**i*p*0.8 + (1-alpha1)**i*s1*0.1 + (1-alpha1)**i*s2*0.1
            down = down + (1-alpha1)**i
        m=2
        itera = i//m
        for k in range(itera):
            sim_score2 = 0.5*cosine_similarity(np.array([cos_proto.iloc[i-m*k].values]),np.array([cos_proto.iloc[i-m-m*k].values]))[0][0] + 0.5*jaccard_score(jac_proto.iloc[i-m*k].values,jac_proto.iloc[i-m-m*k].values)
            session_stat = min(stat_proto['sum'].iloc[i-m*k],stat_proto['sum'].iloc[i-m-m*k])/max(stat_proto['sum'].iloc[i-m*k],stat_proto['sum'].iloc[i-m-m*k])
            cluster_stat = min(stat_proto['sum_cluster'].iloc[i-m*k],stat_proto['sum_cluster'].iloc[i-m-m*k])/max(stat_proto['sum_cluster'].iloc[i-m*k],stat_proto['sum_cluster'].iloc[i-m-m*k])          
            score2.append((sim_score2,session_stat,cluster_stat))    
        if itera>0:        
            for i,(p,s1,s2) in enumerate(score2):
                up = up + (1-alpha2)**i*p*0.8 + (1-alpha2)**i*s1*0.1 + (1-alpha2)**i*s2*0.1
                down = down + (1-alpha2)**i          
        m=3
        itera = i//m
        for k in range(itera):
            sim_score3 = 0.5*cosine_similarity(np.array([cos_proto.iloc[i-m*k].values]),np.array([cos_proto.iloc[i-m-m*k].values]))[0][0] + 0.5*jaccard_score(jac_proto.iloc[i-m*k].values,jac_proto.iloc[i-m-m*k].values)
            session_stat = min(stat_proto['sum'].iloc[i-m*k],stat_proto['sum'].iloc[i-m-m*k])/max(stat_proto['sum'].iloc[i-m*k],stat_proto['sum'].iloc[i-m-m*k])
            cluster_stat = min(stat_proto['sum_cluster'].iloc[i-m*k],stat_proto['sum_cluster'].iloc[i-m-m*k])/max(stat_proto['sum_cluster'].iloc[i-m*k],stat_proto['sum_cluster'].iloc[i-m-m*k])          
            score3.append((sim_score3,session_stat,cluster_stat))    
        if itera>0:        
            for i,p in enumerate(score3):
                up = up + (1-alpha3)**i*p*0.8 + (1-alpha3)**i*s1*0.1 + (1-alpha3)**i*s2*0.1
                down = down + (1-alpha3)**i           
        m=4
        itera = i//m
        for k in range(itera):
            sim_score4 = 0.5*cosine_similarity(np.array([cos_proto.iloc[i-m*k].values]),np.array([cos_proto.iloc[i-m-m*k].values]))[0][0] + 0.5*jaccard_score(jac_proto.iloc[i-m*k].values,jac_proto.iloc[i-m-m*k].values)
            session_stat = min(stat_proto['sum'].iloc[i-m*k],stat_proto['sum'].iloc[i-m-m*k])/max(stat_proto['sum'].iloc[i-m*k],stat_proto['sum'].iloc[i-m-m*k])
            cluster_stat = min(stat_proto['sum_cluster'].iloc[i-m*k],stat_proto['sum_cluster'].iloc[i-m-m*k])/max(stat_proto['sum_cluster'].iloc[i-m*k],stat_proto['sum_cluster'].iloc[i-m-m*k])          
            score4.append((sim_score4,session_stat,cluster_stat))       
        if itera>0:
            for i,p in enumerate(score4):
                up = up + (1-alpha4)**i*p*0.8 + (1-alpha4)**i*s1*0.1 + (1-alpha4)**i*s2*0.1
                down = down + (1-alpha4)**i            
        m=5
        itera = i//m
        for k in range(itera):
            sim_score5 = 0.5*cosine_similarity(np.array([cos_proto.iloc[i-m*k].values]),np.array([cos_proto.iloc[i-m-m*k].values]))[0][0] + 0.5*jaccard_score(jac_proto.iloc[i-m*k].values,jac_proto.iloc[i-m-m*k].values)
            session_stat = min(stat_proto['sum'].iloc[i-m*k],stat_proto['sum'].iloc[i-m-m*k])/max(stat_proto['sum'].iloc[i-m*k],stat_proto['sum'].iloc[i-m-m*k])
            cluster_stat = min(stat_proto['sum_cluster'].iloc[i-m*k],stat_proto['sum_cluster'].iloc[i-m-m*k])/max(stat_proto['sum_cluster'].iloc[i-m*k],stat_proto['sum_cluster'].iloc[i-m-m*k])          
            score5.append((sim_score3,session_stat,cluster_stat))     
        if itera>0:        
            for i,p in enumerate(score5):
                up = up + (1-alpha5)**i*p*0.8 + (1-alpha5)**i*s1*0.1 + (1-alpha5)**i*s2*0.1
                down = down + (1-alpha5)**i          
        m=6
        itera = i//m
        for k in range(itera):
            sim_score6 = 0.5*cosine_similarity(np.array([cos_proto.iloc[i-m*k].values]),np.array([cos_proto.iloc[i-m-m*k].values]))[0][0] + 0.5*jaccard_score(jac_proto.iloc[i-m*k].values,jac_proto.iloc[i-m-m*k].values)
            session_stat = min(stat_proto['sum'].iloc[i-m*k],stat_proto['sum'].iloc[i-m-m*k])/max(stat_proto['sum'].iloc[i-m*k],stat_proto['sum'].iloc[i-m-m*k])
            cluster_stat = min(stat_proto['sum_cluster'].iloc[i-m*k],stat_proto['sum_cluster'].iloc[i-m-m*k])/max(stat_proto['sum_cluster'].iloc[i-m*k],stat_proto['sum_cluster'].iloc[i-m-m*k])          
            score6.append((sim_score6,session_stat,cluster_stat))     
        if itera>0:        
            for i,p in enumerate(score6):
                up = up + (1-alpha6)**i*p*0.8 + (1-alpha6)**i*s1*0.1 + (1-alpha6)**i*s2*0.1
                down = down + (1-alpha6)**i          
        final_score = up/down
        score_dict[date] = final_score
    proto_score_dict[proto] = score_dict
score_dict

100%|██████████| 11/11 [00:00<00:00, 19.94it/s]


{'20200107': 0.6541072356622817,
 '20200108': 0.7029105025003434,
 '20200109': 0.7023759937950809,
 '20200110': 0.6173728461777567,
 '20200111': 0.5878179853618903,
 '20200112': 0.6101793114825371}

In [185]:
# proto_score_dict['smb']

# kk = cosine_similarity(np.array([cos_proto.iloc[i-k].values]),np.array([cos_proto.iloc[i-1-k].values]))[0][0]
# kk

# proto_df.loc[:,'sum']

# proto_df.loc[:,proto_df.columns[:-1]]/proto_df.loc[:,'sum']

# # temp.append( pd.Series(temp.sum().T,name='sum'))
# # temp
# temp.loc[:,'sum'] = temp.sum(axis=1)
# temp.loc["sum",:] = temp.sum(axis=0)
# temp

# for proto in tqdm(proto_li):
#     proto_df = date_proto_df[proto]
    

{'20200107': 0.8564295914911586,
 '20200108': 0.8864001180863158,
 '20200109': 0.8796556550007797,
 '20200110': 0.8731084420241771,
 '20200111': 0.8327381137880794,
 '20200112': 0.830946281919998}